## FEC Campaign Finance

### Committee Master

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime as dt

In [2]:
year = '2020'

In [3]:
# read in & clean
# https://www.fec.gov/files/bulk-downloads/2018/cm18.zip
cm = pd.read_csv(f'data/cm{year[2:]}.zip', sep = '|', error_bad_lines = False, header = None)
print('original length:', len(cm))

# add column headers from separate file
headers = pd.read_csv('data/cm_header_file.csv')
cm.columns = [x.lower() for x in headers.columns]

# preserve uncleaned tres_nm & cmte_st1
cm['orig_tres_nm'] = list(cm['tres_nm'].values)
cm['orig_cmte_st1'] = list(cm['cmte_st1'].values)

# prep strings for deduping
change_dict = {
    'STREET' : 'ST',
    'AVENUE' : 'AVE',
    ' DRIVE' : ' DR',
    ' ROAD' : ' RD',
    ' SUITE' : ' STE',
    '.' : '',
    ',' : '',
    'BOULEVARD' : 'BLVD',
    ' PLACE' : ' PL',
    
}
keys = list(set(change_dict.keys()))
for key in keys:
    cm['cmte_st1'] = [str(x).replace(key, change_dict[key]) for x in cm['cmte_st1']]

keep_0 = [
    ' STE ',
    '#',
    ' ST ',
    ' AVE ',
    ' RD ',
    ' PL ', 
    ' BLVD ', 
    ' DR ',
]

cardinals = [
    'NW',
    'NE',
    'SW',
    'SE',
]
cm['cmte_st1'] = [x.split(' STE ')[0].split('#')[0] for x in cm['cmte_st1']]
cm['cmte_st1'] = [' '.join(str(x).split()) for x in cm['cmte_st1']]
# create full address column from cleaned parts
cm['address'] = cm['cmte_st1'] + ', ' + cm['cmte_city'] + ', ' + cm['cmte_st']

cm.head(2)

original length: 15294


,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
0,C00000059,HALLMARK CARDS PAC,SARAH MOE,2501 MCGEE,MD #500,KANSAS CITY,MO,64108,U,Q,UNK,M,C,NaN,NaN,SARAH MOE,2501 MCGEE,"2501 MCGEE, KANSAS CITY, MO"
1,C00000422,AMERICAN MEDICAL ASSOCIATION POLITICAL ACTION ...,"WALKER, KEVIN MR.",25 MASSACHUSETTS AVE NW,SUITE 600,WASHINGTON,DC,200017400,B,Q,NaN,M,M,DELAWARE MEDICAL PAC,NaN,"WALKER, KEVIN MR.","25 MASSACHUSETTS AVE, NW","25 MASSACHUSETTS AVE NW, WASHINGTON, DC"


In [4]:
# check for duplicate rows
cm[cm.duplicated(keep = False)]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
7738,C00654699,THE COMMITTEE TO ELECT JUAN PARKS,"MCMILLAN, EVA MRS",6067 FAIRINGTON FARMS LANE,NaN,LITHONIA,GA,300381547,P,H,DEM,A,NaN,NaN,H8GA04117,"MCMILLAN, EVA MRS",6067 FAIRINGTON FARMS LANE,"6067 FAIRINGTON FARMS LANE, LITHONIA, GA"
7739,C00654699,THE COMMITTEE TO ELECT JUAN PARKS,"MCMILLAN, EVA MRS",6067 FAIRINGTON FARMS LANE,NaN,LITHONIA,GA,300381547,P,H,DEM,A,NaN,NaN,H8GA04117,"MCMILLAN, EVA MRS",6067 FAIRINGTON FARMS LANE,"6067 FAIRINGTON FARMS LANE, LITHONIA, GA"
10276,C00690123,INDEPENDENT VOICES OF CHANGE,"PHELAN, JOHN",2 MEGHANS WAY,NaN,LYNN,MA,01904,U,O,NaN,T,NaN,NaN,H8NJ07256,"PHELAN, JOHN",2 MEGHANS WAY,"2 MEGHANS WAY, LYNN, MA"
10277,C00690123,INDEPENDENT VOICES OF CHANGE,"PHELAN, JOHN",2 MEGHANS WAY,NaN,LYNN,MA,01904,U,O,NaN,T,NaN,NaN,H8NJ07256,"PHELAN, JOHN",2 MEGHANS WAY,"2 MEGHANS WAY, LYNN, MA"


In [5]:
# drop duplicates
cm.drop_duplicates(keep = 'first', inplace = True)

In [6]:
# see which cmte_ids appear more than once
cm['cmte_id'].value_counts()[cm['cmte_id'].value_counts() > 1]

Series([], Name: cmte_id, dtype: int64)

In [7]:
# check for nulls
cm.isnull().sum()[cm.isnull().sum() > 0].sort_values()

cmte_nm                     2
cmte_tp                     4
cmte_st                     7
cmte_dsgn                   7
cmte_city                   8
orig_cmte_st1               8
address                     8
cmte_zip                   10
tres_nm                   500
orig_tres_nm              500
connected_org_nm         6165
cmte_pty_affiliation     8549
cand_id                  9585
cmte_st2                11612
org_tp                  12064
dtype: int64

In [8]:
# see which have missing cmte_nm
cm[cm['cmte_nm'].isnull()]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
8691,C00669044,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN
9312,C00677690,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,H8PA13158,NaN,NaN,NaN


In [9]:
# fill in placeholder cmte_nm
cm.loc[cm['cmte_nm'].isnull(), 'cmte_nm'] = '(cmte_nm missing)'

In [10]:
# see which have missing address
# then see if any other rows have matching cmte_nm
cm[cm['address'].isnull()]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
6087,C00608414,NANA FOR SENATE,NANA TCHIENKOU,nan,NaN,NaN,NaN,NaN,P,S,NaN,Q,NaN,NaN,NaN,NANA TCHIENKOU,NaN,NaN
8691,C00669044,(cmte_nm missing),NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN
9248,C00676767,DERRICK MICHAEL REID,NaN,nan,NaN,NaN,NaN,NaN,P,S,NaN,A,NaN,NaN,S8CA00507,NaN,NaN,NaN
9312,C00677690,(cmte_nm missing),NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,H8PA13158,NaN,NaN,NaN
10286,C00690214,FREEDOM FOR ALL 777,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN
14798,C30002943,BLACK VOTERS MATTER FUND INC,NaN,nan,NaN,NaN,NaN,NaN,U,E,NaN,Q,NaN,NaN,NaN,NaN,NaN,NaN
15210,C90018243,SOMOS ACCION,NaN,nan,NaN,NaN,NaN,NaN,U,I,NaN,Q,NaN,NaN,NaN,NaN,NaN,NaN
15274,C90018912,AVALANCHE MARKETING,NaN,6244 OAKLAND AVE,STE. 303,NaN,MO,NaN,U,I,NaN,Q,NaN,NaN,NaN,NaN,6244 OAKLAND AVE.,NaN


In [11]:
cm[cm['cmte_nm'].apply(lambda x: 'NANA' in x)]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
4414,C00512327,NANA REGIONAL CORPORATION PAC (NANAPAC),"BOLTON, JEFFERY D",909 W 9TH AVE,NaN,ANCHORAGE,AK,99501,B,Q,NaN,Q,C,NANA REGIONAL CORPORATION,NaN,"BOLTON, JEFFERY D",909 W. 9TH AVE,"909 W 9TH AVE, ANCHORAGE, AK"
6087,C00608414,NANA FOR SENATE,NANA TCHIENKOU,nan,NaN,NaN,NaN,NaN,P,S,NaN,Q,NaN,NaN,NaN,NANA TCHIENKOU,NaN,NaN


In [12]:
cm[cm['cmte_nm'].apply(lambda x: 'DERRICK' in x)]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
6973,C00636613,DERRICK CROWE FOR CONGRESS COMMITTEE,"CROWE, LAURIE MS.",10932 MINT JULEP DR,NaN,AUSTIN,TX,78748,P,H,DEM,T,NaN,NaN,H8TX21158,"CROWE, LAURIE MS.",10932 MINT JULEP DR,"10932 MINT JULEP DR, AUSTIN, TX"
7844,C00656538,DERRICK LADARIUS EGGLESTON,DERRICK EGGLESTON,7087 MARKIM DR,NaN,BARTLETT,TN,38133,P,P,UNK,Q,NaN,NaN,P00006338,DERRICK EGGLESTON,7087 MARKIM DR,"7087 MARKIM DR, BARTLETT, TN"
9248,C00676767,DERRICK MICHAEL REID,NaN,nan,NaN,NaN,NaN,NaN,P,S,NaN,A,NaN,NaN,S8CA00507,NaN,NaN,NaN
10997,C00698266,DERRICK WADELL COOPER,"COOPER, DERRICK WADELL",GENERAL DELIVERY,NaN,VISALIA,CA,93291,A,P,APF,Q,NaN,NONE,P00010538,"COOPER, DERRICK WADELL",GENERAL DELIVERY,"GENERAL DELIVERY, VISALIA, CA"


In [13]:
cm[cm['cmte_nm'].apply(lambda x: '777' in x)]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
10286,C00690214,FREEDOM FOR ALL 777,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# this one matches... they might've filed wrong cmte_tp at first then redone
cm[cm['cmte_nm'].apply(lambda x: 'BLACK VOTER' in x)]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
14798,C30002943,BLACK VOTERS MATTER FUND INC,NaN,nan,NaN,NaN,NaN,NaN,U,E,NaN,Q,NaN,NaN,NaN,NaN,NaN,NaN
15125,C90017385,BLACK VOTERS MATTER FUND INC,NaN,3390 STONEWALL TELL RD,NaN,ATLANTA,GA,30349,U,I,NaN,Q,NaN,NaN,NaN,NaN,3390 STONEWALL TELL RD,"3390 STONEWALL TELL RD, ATLANTA, GA"


In [15]:
cm[cm['cmte_nm'].apply(lambda x: 'SOMOS' in x)]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
13476,C00724203,SOMOS PAC,"MORALES, MELISSA",700 13TH ST NW,SUITE 600,WASHINGTON,DC,20005,U,O,NaN,Q,NaN,NONE,NaN,"MORALES, MELISSA","700 13TH STREET, NW","700 13TH ST NW, WASHINGTON, DC"
15210,C90018243,SOMOS ACCION,NaN,nan,NaN,NaN,NaN,NaN,U,I,NaN,Q,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
cm[cm['cmte_nm'].apply(lambda x: 'AVALANCHE' in x)]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
15274,C90018912,AVALANCHE MARKETING,NaN,6244 OAKLAND AVE,STE. 303,NaN,MO,NaN,U,I,NaN,Q,NaN,NaN,NaN,NaN,6244 OAKLAND AVE.,NaN


In [17]:
# impute city
cm.loc[cm['cmte_nm'].apply(lambda x: 'AVALANCHE' in x), 'cmte_city'] = 'ST LOUIS'

In [18]:
# redo address column
cm['address'] = cm['cmte_st1'] + ', ' + cm['cmte_city'] + ', ' + cm['cmte_st']

In [19]:
# see which street addresses appear most frequently
cm['address'].value_counts()[:20]

228 S WASHINGTON ST, ALEXANDRIA, VA        166
PO BOX 30844, BETHESDA, MD                 112
824 S MILLEDGE AVE, ATHENS, GA              95
PO BOX 26141, ALEXANDRIA, VA                93
918 PENNSYLVANIA AVE SE, WASHINGTON, DC     93
PO BOX 60148, WASHINGTON, DC                59
700 13TH ST NW, WASHINGTON, DC              56
611 PENNSYLVANIA AVE SE, WASHINGTON, DC     53
610 S BLVD, TAMPA, FL                       51
910 17TH ST NW, WASHINGTON, DC              48
777 S FIGUEROA ST, LOS ANGELES, CA          44
PO BOX 9891, ARLINGTON, VA                  41
499 SOUTH CAPITOL ST SW, WASHINGTON, DC     37
PO BOX 15320, WASHINGTON, DC                35
5429 MADISON AVE, SACRAMENTO, CA            34
249 E OCEAN BLVD, LONG BEACH, CA            33
PO BOX 97275, RALEIGH, NC                   31
555 CAPITOL MALL, SACRAMENTO, CA            28
C/O RED CURVE SOLUTIONS, BEVERLY, MA        28
901 N WASHINGTON ST, ALEXANDRIA, VA         26
Name: address, dtype: int64

In [20]:
# fill in placeholder for missing treasurer name 
print(len(cm[cm['tres_nm'].isnull()]))
cm.loc[cm['tres_nm'].isnull(), 'tres_nm'] = '(no treasurer listed)'
cm.loc[cm['tres_nm'] == '.', 'tres_nm'] = '(no treasurer listed)'
cm.loc[cm['tres_nm'] == 'NONE', 'tres_nm'] = '(no treasurer listed)'
cm.loc[cm['tres_nm'] == '', 'tres_nm'] = '(no treasurer listed)'

# convert to <first name> <last name> format from <last name>, <first name>
cm['tres_nm'] = [' '.join([' '.join(x.split(', ')[1:]), x.split(', ')[0]]) if ', ' in \
                 x else x for x in cm['tres_nm']]
# clean up spacing
# looks like this could just be ' '.join(x.strip()) 
# but i feel like i did it this way for a reason... leaving it
cm['tres_nm'] = [' '.join([x for x in list(x.split(' ')) if len(x) > 1]).strip() for \
                 x in cm['tres_nm']]

# prep for deduping

# substitutions
subs = {
    'XAVUER' : 'XAVIER',
    'JSOEPH' : 'JOSEPH',
    'WILLAIM' : 'WILLIAM',
    'V?RONIQUE' : 'VERONIQUE',
    '.' : '',
    ',' : '',
    
}
for key in list(subs.keys()):
    cm['tres_nm'] = [x.replace(key, subs[key]) for x in cm['tres_nm']]

# drops
drops = [
    'MRS',
    'MS',
    'MR',
    'MISS',
    'JR',
    'SR',
    'UNDEFINED',
    'JD',
    'CPA',
    'HONORABLE',
    'FED',
    'II', 
    'III',
    'IV',
    'ESQ',
    'DR',
    'MD',
    'DO',
    'CFO',
    'CEO',
    'CTO',
    'TREAS',
    'US',
    '1970',
    '--SELECT',
    '1973',
    '1979',
    
]
for droplet in drops:
    cm['tres_nm'] = [' '.join([x for x in x.split() if \
                               ((x not in drops) & (len(x) > 1))]) for x in cm['tres_nm']]

# a bunch of entries have last name appearing twice; 
# check for this and keep only first instance
cm['tres_nm'] = [x.split() for x in cm['tres_nm']]
def f7(seq):
    '''https://stackoverflow.com/questions/480214/\
    how-do-you-remove-duplicates-from-a-list-whilst-preserving-order'''
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]            
cm['tres_nm'] = [' '.join(f7(x)) for x in cm['tres_nm']]

# see tres_nms appearing most frequently
cm['tres_nm'].value_counts()[:20]

500


(no treasurer listed)    505
PAUL KILGORE             164
LISA LISKER              121
CHRIS MARSTON            114
STEVEN MARTIN            109
BRADLEY CRATE             95
THOMAS DATWYLER           77
JUDITH ZAMORE             75
JAY PETTERSON             73
JENNIFER MAY              73
LES WILLIAMSON            68
JANICA KYRIACOPOULOS      67
NANCY WATKINS             65
CABELL HOBBS              65
DAVID SATTERFIELD         58
ELIZABETH CURTIS          51
KEITH DAVIS               42
ROBERT PHILLIPS           42
STEVEN MELE               38
SUE JACKSON               37
Name: tres_nm, dtype: int64

In [21]:
# what is street address of committees not listing treasurer
cm.loc[cm['tres_nm'] == '(no treasurer listed)', 'address'].value_counts()[:20]

1201 CONNECTICUT AVE NW, WASHINGTON, DC      9
700 13TH ST NW, WASHINGTON, DC               7
555 CAPITOL MALL, SACRAMENTO, CA             4
1875 CONNECTICUT AVE NW, WASHINGTON, DC      3
125 BROAD ST, NEW YORK, NY                   3
PO BOX 26141, ALEXANDRIA, VA                 3
625 BROAD ST, NEWARK, NJ                     3
1120 CONNECTICUT AVE NW, WASHINGTON, DC      3
815 16TH ST NW, WASHINGTON, DC               3
1747 PENNSYLVANIA AVE NW, WASHINGTON, DC     3
1800 MASSACHUSETTS AVE NW, WASHINGTON, DC    2
228 S WABASH, CHICAGO, IL                    2
119 1ST AVE S S320, SEATTLE, WA              2
1225 S WELLER ST, SEATTLE, WA                2
2001 L ST NE, WASHINGTON, DC                 2
1319 GREEN ST, HARRISBURG, PA                2
10 E DOTY ST, MADISON, WI                    2
5429 MADISON AVE, SACRAMENTO, CA             2
213 DONCASTER RD, ARNOLD, MD                 2
700 2ND ST NE, WASHINGTON, DC                2
Name: address, dtype: int64

In [22]:
list(cm.loc[cm['address'] == '1201 CONNECTICUT AVE NW, WASHINGTON, DC', 'cmte_nm'])

['CONSTITUTIONAL RESPONSIBILITY PROJECT',
 'SOCAL HEALTH CARE COALITION A PROJECT OF SIXTEEN THIRTY FUND',
 'FLORIDIANS FOR A FAIR SHAKE A PROJECT OF SIXTEEN THIRTY FUND',
 'DEMAND JUSTICE A PROJECT OF SIXTEEN THIRTY FUND',
 'SIXTEEN THIRTY FUND/MAKE IT WORK ACTION',
 'SIXTEEN THIRTY FUND/NOT ONE PENNY',
 'FLORIDIANS FOR A FAIR SHAKE A PROJECT OF SIXTEEN THIRTY FUND',
 'SOCAL HEALTH CARE COALITION A PROJECT OF SIXTEEN THIRTY FUND',
 'OHIOANS FOR ECONOMIC OPPORTUNITY A PROJECT OF SIXTEEN THIRTY FUND']

In [23]:
list(cm.loc[
    (cm['address'] == '700 13TH ST NW, WASHINGTON, DC') & \
    (cm['tres_nm'] == '(no treasurer listed)'), 'cmte_nm'])

['MAJORITY FORWARD',
 'MAJORITY FORWARD',
 'AMERICA WORKING TOGETHER',
 'DUTY AND HONOR',
 'BLACK PROGRESSIVE ACTION COALITION',
 'BLACK ECONOMIC ALLIANCE FUND',
 'HOUSE MAJORITY FORWARD']

In [24]:
list(cm.loc[
    (cm['address'] == '555 CAPITOL MALL, SACRAMENTO, CA') & \
    (cm['tres_nm'] == '(no treasurer listed)'), 'cmte_nm'])

['PLANNED PARENTHOOD ADVOCACY PROJECT LOS ANGELES COUNTY',
 'ALAMEDANS UNITED SUPPORTING VELLA AND ASHCRAFT FOR CITY COUNCIL, ET AL',
 'OPPORTUNITY PAC - A COALITION OF TEACHERS HEALTH CARE GIVERS FACULTY MEMBERS SCHOOL EMPLOYEES AND PUBLIC AND PR',
 'GOLDEN CALIFORNIA COMMITTEE SPONSORED BY THE SEIU CALIFORNIA STATE COUNCIL']

### Dedupe treasurer names

This is a journey.

In [25]:
def findall(name):
    '''find pattern in treasurer names columns'''
    all_tres = list(set(cm['tres_nm']))
    if type(name) == str:
        these = [x for x in all_tres if name.lower() in str(x).lower()]
    else:  
        these = all_tres
        for which in name:
            these = [x for x in these if which in str(x).lower()]
    return sorted(these)

In [26]:
def getall(these, new):
    '''assign deduped treasurer name'''
    print('orig len:', len(cm[cm['tres_nm'] == new]))
    for which in these:
        cm.loc[cm['tres_nm'] == which, 'tres_nm'] = new
    print('new len:', len(cm[cm['tres_nm'] == new]))
    return 

In [27]:
# this list keeps track of the ones we've already addressed
dones = ['(no treasurer listed)']

In [28]:
findall(' kil')

['BRIAN KILDEE',
 'CLIFTON JULIUS KILBY',
 'GERALD KILPATRICK',
 'JEFFREY TUTTLE KILLEEN',
 'KENDAL KILLIAN',
 'KEVIN KILEY',
 'LAWRENCE KILGORE',
 'MEAGHAN KILLION JOYCE',
 'MILES KILCOIN',
 'PAUL KILGORE',
 'RACHEL KILPATRICK',
 'ROBIN KILEY',
 'TERRY KILROY',
 'WILLIAM KILLMER']

In [29]:
findall(' joyce')

['CASEY JOYCE',
 'ELAINE JOYCE MCKINNEY',
 'KATHY JOYCE STREUSAND',
 'MEAGHAN KILLION JOYCE']

In [30]:
dones.extend(findall(' kil'))

In [31]:
findall(' lis')

['(no treasurer listed)',
 'DOUGLASS LIST',
 'EVAN ROGER LISTOPAD',
 'JACKSON LISLE',
 'JOHN LISTAK',
 'LISA LISKER',
 'PAMELA DE LISLE',
 'SUSIE LISA']

In [32]:
dones.extend(findall(' lis'))

In [33]:
findall('lisker')

['LISA LISKER', 'LISKERLISA']

In [34]:
new = 'LISA LISKER'
dones.append(new)

these = findall('lisker')

getall(these, new)

orig len: 121
new len: 122


In [35]:
findall(['c', 'marst'])

['CHRIS MARSTEN', 'CHRIS MARSTON', 'CHRISTOPHER MARSTON']

In [36]:
cm[cm['tres_nm'] == 'CHRIS MARSTEN']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
8512,C00666511,WE STAND FOR BETTER,CHRIS MARSTEN,PO BOX 26141,NaN,ALEXANDRIA,VA,22313,U,O,NaN,T,NaN,NaN,NaN,CHRIS MARSTEN,PO BOX 26141,"PO BOX 26141, ALEXANDRIA, VA"


In [37]:
cm[cm['tres_nm'] == 'CHRIS MARSTON'].head()

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
546,C00105338,"FOLEY & LARDNER POLITICAL FUND, INC.",CHRIS MARSTON,3000 K ST NW,NaN,WASHINGTON,DC,20007,B,Q,NaN,M,M,NONE,NaN,"MARSTON, CHRIS",3000 K STREET NW,"3000 K ST NW, WASHINGTON, DC"
2283,C00372201,REDWINGMCPAC,CHRIS MARSTON,PO BOX 26141,NaN,ALEXANDRIA,VA,22313,U,N,REP,Q,NaN,NONE,H2PA18143,"MARSTON, CHRIS",PO BOX 26141,"PO BOX 26141, ALEXANDRIA, VA"
2799,C00410118,BACHMANN FOR CONGRESS,CHRIS MARSTON,PO BOX 26141,NaN,ALEXANDRIA,VA,22313,P,H,REP,Q,NaN,BACHMANN FOR PRESIDENT,H6MN06074,"MARSTON, CHRIS",PO BOX 26141,"PO BOX 26141, ALEXANDRIA, VA"
3180,C00433524,DUNCAN D. HUNTER FOR CONGRESS,CHRIS MARSTON,PO BOX 1545,NaN,EL CAJON,CA,92022,P,H,REP,Q,NaN,HUNTER VICTORY FUND,H8CA52052,"MARSTON, CHRIS",PO BOX 1545,"PO BOX 1545, EL CAJON, CA"
3288,C00441014,ROB WITTMAN FOR CONGRESS,CHRIS MARSTON,PO BOX 427,NaN,ALEXANDRIA,VA,22313,P,H,REP,Q,NaN,WITTMAN VICTORY COMMITTEE,H8VA01147,"MARSTON, CHRIS",PO BOX 427,"PO BOX 427, ALEXANDRIA, VA"


In [38]:
new = 'CHRISTOPHER MARSTON'
dones.append(new)

these = findall(['c', 'marst'])

getall(these, new)

orig len: 8
new len: 123


In [39]:
findall(['st', ' martin'])

['ERNESTO MARTINEZ',
 'JULIAN BAUTISTA MARTINEZ',
 'STEVE MARTIN',
 'STEVEN MARTIN']

In [40]:
cm[cm['tres_nm'] == 'STEVE MARTIN']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
14289,C00732685,BURRIL FOR CONGRESS,STEVE MARTIN,PO BOX 1039,NaN,SKYLAND,NC,28776,P,H,REP,Q,NaN,NONE,H0NC11217,"MARTIN, STEVE",P.O. BOX 1039,"PO BOX 1039, SKYLAND, NC"


In [41]:
cm[cm['tres_nm'] == 'STEVEN MARTIN'].head()

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
1280,C00233353,AD ALLIANCE,STEVEN MARTIN,PO BOX 30844,NaN,BETHESDA,MD,20824,B,Q,NaN,M,M,NONE,NaN,"MARTIN, STEVEN JR.",PO BOX 30844,"PO BOX 30844, BETHESDA, MD"
2286,C00372532,MICHAEL BURGESS FOR CONGRESS,STEVEN MARTIN,PO BOX 2334,NaN,DENTON,TX,762022334,P,H,REP,Q,NaN,CARE AMERICA,H2TX26093,"MARTIN, STEVEN G. JR.",PO BOX 2334,"PO BOX 2334, DENTON, TX"
2900,C00415208,LONE STAR LEADERSHIP PAC,STEVEN MARTIN,PO BOX 30844,NaN,BETHESDA,MD,208240844,D,Q,NaN,M,NaN,MICHAEL BURGESS,NaN,"MARTIN, STEVEN",PO BOX 30844,"PO BOX 30844, BETHESDA, MD"
3338,C00444620,FRIENDS OF GLENN THOMPSON,STEVEN MARTIN,400 N MICHAEL ST,NaN,ST. MARYS,PA,15857,P,H,REP,Q,NaN,PROBLEM SOLVERS PATRIOTS,H8PA05071,"MARTIN, STEVEN",400 N. MICHAEL STREET,"400 N MICHAEL ST, ST. MARYS, PA"
3776,C00475103,FITZPATRICK FOR CONGRESS,STEVEN MARTIN,PO BOX 185,NaN,LANGHORNE,PA,190470185,P,H,REP,Q,NaN,NONE,H4PA08074,"MARTIN, STEVEN G. JR.",PO BOX 185,"PO BOX 185, LANGHORNE, PA"


In [42]:
dones.extend(findall(['st', ' martin']))

In [43]:
findall(' crat')

['BRADLEY CRATE', 'VIVIAN CRATER']

In [44]:
dones.extend(findall(' crat'))

In [45]:
findall('datwyler')

['DATWYLER THOMAS',
 'THOMAS CHARLES DATWYLER',
 'THOMAS DATWYLER',
 'TOM DATWYLER']

In [46]:
new = 'THOMAS DATWYLER'
dones.append(new)

these = findall('datwyler')

getall(these, new)

orig len: 77
new len: 80


In [47]:
findall('zamore')

['JUDITH ZAMORE', 'JUDY ZAMORE']

In [48]:
new = 'JUDITH ZAMORE'
dones.append(new)

these = findall('zamore')

getall(these, new)

orig len: 75
new len: 77


In [49]:
findall(' pett')

['AARON PETTIGREW', 'DEREK LEONARD PETTEYS', 'JAY PETTERSON']

In [50]:
dones.extend(findall(' pett'))

In [51]:
findall(['jen', 'may'])

['JENNIFER MAY']

In [52]:
dones.append('JENNIFER MAY')

In [53]:
# first jennifer for driskell
cm[cm['tres_nm'] == 'JENNIFER FAIRFIELD']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
5488,C00572859,GRETCHEN DRISKELL FOR CONGRESS,JENNIFER FAIRFIELD,PO BOX 464,NaN,SALINE,MI,48176,P,H,DEM,T,NaN,NaN,H6MI07223,"FAIRFIELD, JENNIFER",PO BOX 464,"PO BOX 464, SALINE, MI"


In [54]:
# second jennifer for driskell.  same?
cm[(cm['tres_nm'] == 'JENNIFER MAY') & (cm['cmte_st1'] == 'PO BOX 464')]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
14026,C00729947,GRETCHEN DRISKELL FOR CONGRESS,JENNIFER MAY,PO BOX 464,NaN,SALINE,MI,48176,P,H,DEM,Q,NaN,NONE,H0MI07150,"MAY, JENNIFER",PO BOX 464,"PO BOX 464, SALINE, MI"


In [55]:
findall(' williamson')

['CRAIG WILLIAMSON',
 'JAMES WILLIAMSON',
 'JOSHUA WILLIAMSON',
 'LES WILLIAMSON',
 'SEAN WILLIAMSON']

In [56]:
dones.extend(findall(' williamson'))

In [57]:
findall(' ky')

['JANICA KYRIACOPOULOS', 'SAMMY KYE']

In [58]:
dones.extend(findall(' ky'))

In [59]:
findall(' watkins')

['BILL WATKINS',
 'CHRISTOPHER CHRISTIAN WATKINS',
 'ERIK WATKINS',
 'NANCY WATKINS',
 'WILLIAM WATKINS']

In [60]:
cm[cm['tres_nm'] == 'BILL WATKINS']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
13967,C00729350,KUSTOFF VICTORY FUND,BILL WATKINS,1661 AARON BRENNER DR,SUITE 300,MEMPHIS,TN,38120,J,N,NaN,Q,NaN,NONE,NaN,"WATKINS, BILL",1661 AARON BRENNER DR,"1661 AARON BRENNER DR, MEMPHIS, TN"


In [61]:
cm[cm['tres_nm'] == 'WILLIAM WATKINS']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
2322,C00376244,KUSTOFF FOR CONGRESS 2002,WILLIAM WATKINS,1661 AARON BRENNER DR,SUITE 300,MEMPHIS,TN,381201466,A,H,REP,Q,NaN,NONE,H2TN07103,"WATKINS, WILLIAM H JR",1661 AARON BRENNER DRIVE,"1661 AARON BRENNER DR, MEMPHIS, TN"
6074,C00607937,KELSEY FOR CONGRESS,WILLIAM WATKINS,110 E MULBERRY ST,SUITE 200,COLLIERVILLE,TN,38017,P,H,REP,Q,NaN,NONE,H6TN08151,"WATKINS, WILLIAM H. III",110 E. MULBERRY STREET,"110 E MULBERRY ST, COLLIERVILLE, TN"
6243,C00614826,KUSTOFF FOR CONGRESS,WILLIAM WATKINS,1661 AARON BRENNER DR,STE 300,MEMPHIS,TN,381201466,P,H,REP,Q,NaN,NaN,H2TN07103,"WATKINS, WILLIAM H JR",1661 AARON BRENNER DR,"1661 AARON BRENNER DR, MEMPHIS, TN"


In [62]:
new = 'WILLIAM WATKINS'
dones.append(new)

these = ['BILL WATKINS', 'WILLIAM WATKINS']

getall(these, new)

orig len: 3
new len: 4


In [63]:
dones.extend(findall(' watkins'))

In [64]:
findall('hobbs')

['ALLISON HOBBS GILES', 'CABELL HOBBS', 'NICOLE HOBBS', 'SCOTT HOBBS']

In [65]:
findall('giles')

['ALLISON HOBBS GILES', 'JASON GILES', 'LAWSON GILES']

In [66]:
dones.extend(findall('giles'))

In [67]:
dones.extend(findall('hobbs'))

In [68]:
findall(' satt')

['ADAM SATTERFIELD', 'DAVID SATTERFIELD', 'STACEY SATTERLEE']

In [69]:
dones.extend(findall(' satt'))

In [70]:
findall(' curtis')

['BOBBY CURTIS BRACKETT',
 'CLINT CURTIS',
 'DAVID CURTIS',
 'ELIZABETH CURTIS',
 'KEVIN CURTIS',
 'LIZ CURTIS',
 'MICHAEL CURTIS DEAN',
 'RON CURTIS',
 'STEVEN DALE CURTIS']

In [71]:
findall('brackett')

['BOBBY CURTIS BRACKETT']

In [72]:
findall(['mi', 'dean'])

['MICHAEL CURTIS DEAN']

In [73]:
new = 'ELIZABETH CURTIS'
dones.append(new)

these = findall(['liz', 'curtis'])

getall(these, new)

orig len: 51
new len: 55


In [74]:
dones.extend(findall(' curtis'))

In [75]:
findall(' davis')

['ADAM DAVIS',
 'BILLY DAVIS',
 'BROOKE DAVIS',
 'CHERISH FOREMAN DAVIS',
 'CLAY PARKER DAVIS',
 'CYNTHIA DAVIS',
 'DEBORAH DAVIS',
 'DONALD DAVIS',
 'GARY DAVIS',
 'HAROLD DAVIS',
 'JACOB DAVIS',
 'JAMES DAVIS',
 'JOE DAVIS',
 'KAYOLKA DAVIS',
 'KEITH DAVIS',
 'KELLY DAVIS',
 'KETH DAVIS',
 'LANCE DAVIS',
 'MARIENELLA DAVIS',
 'MICHAEL DAVIS',
 'MICHELE DAVISON',
 'MIKE DAVIS',
 'MILTON DAVIS',
 'RACHEL DAVIS',
 'REGINALD DAVIS',
 'RICK DAVISON',
 'SAMANTHA DAVIS',
 'SANDRA DAVIS',
 'TANYA DAVIS',
 'VANDY DAVIS',
 'VERA DAVIS',
 'WAYNE DAVIS',
 'WILLIAM DAVIS']

In [76]:
cm[cm['tres_nm'].apply(lambda x: x in ['BILLY DAVIS', 'WILLIAM DAVIS'])]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
6208,C00613216,THE CANDIDATE BILLY DAVIS INC.,BILLY DAVIS,1455 WIMBLEDON DR,NaN,KENNESAW,GA,30144,P,H,REP,Q,NaN,NaN,H6GA11074,"DAVIS, BILLY",1455 WIMBLEDON DRIVE,"1455 WIMBLEDON DR, KENNESAW, GA"
7088,C00638536,FRIENDS OF DANNER KLINE,WILLIAM DAVIS,PO BOX 430191,NaN,VESTAVIA HILLS,AL,35243,P,H,DEM,Q,NaN,NONE,H8AL06099,"DAVIS, WILLIAM",PO BOX 430191,"PO BOX 430191, VESTAVIA HILLS, AL"


In [77]:
cm[cm['tres_nm'] == 'KETH DAVIS']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
8423,C00665232,WIN IN 2018,KETH DAVIS,228 S WASHINGTON ST,SUITE 115,ALEXANDRIA,VA,22314,J,N,NaN,T,NaN,NaN,NaN,"DAVIS, KETH A.",228 S. WASHINGTON STREET,"228 S WASHINGTON ST, ALEXANDRIA, VA"


In [78]:
cm[cm['tres_nm'] == 'KEITH DAVIS'].head()

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
206,C00027466,NRSC,KEITH DAVIS,425 2ND ST NE,NaN,WASHINGTON,DC,20002,U,Y,REP,M,NaN,2018 TILLIS SENATE CANDIDATE FUND,S4AK00214,"DAVIS, KEITH",425 2ND STREET NE,"425 2ND ST NE, WASHINGTON, DC"
383,C00075820,NRCC,KEITH DAVIS,320 FIRST ST SE,.,WASHINGTON,DC,20003,U,Y,REP,M,NaN,COLE COMBINED COMMITTEE,H4WA04104,"DAVIS, KEITH A.",320 FIRST STREET SE,"320 FIRST ST SE, WASHINGTON, DC"
1853,C00330720,TRUST PAC TEAM REPUBLICANS FOR UTILIZING SENSI...,KEITH DAVIS,228 S WASHINGTON ST,SUITE 115,ALEXANDRIA,VA,22314,D,Q,NaN,Q,NaN,NONE,NaN,"DAVIS, KEITH A.",228 S. WASHINGTON STREET,"228 S WASHINGTON ST, ALEXANDRIA, VA"
2423,C00383745,ALEXANDER FOR SENATE 2020 INC,KEITH DAVIS,228 S WASHINGTON ST,NaN,ALEXANDRIA,VA,223145404,P,S,REP,Q,NaN,NaN,S2TN00058,KEITH A DAVIS,228 S WASHINGTON STREET SUITE 115,"228 S WASHINGTON ST, ALEXANDRIA, VA"
2493,C00388421,TENN POLITICAL ACTION COMMITTEE INC (TENN PAC),KEITH DAVIS,228 S WASHINGTON ST,NaN,ALEXANDRIA,VA,22314,D,Q,NaN,Q,NaN,NONE,NaN,"DAVIS, KEITH A.",228 S WASHINGTON STREET SUITE 115,"228 S WASHINGTON ST, ALEXANDRIA, VA"


In [79]:
new = 'KEITH DAVIS' 
dones.append(new)

these = ['KEITH DAVIS', 'KETH DAVIS']

getall(these, new)

orig len: 42
new len: 43


In [80]:
cm[cm['tres_nm'].apply(lambda x: x in ['MIKE DAVIS', 'MICHAEL DAVIS'])]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
8534,C00666826,JEFF MILLER FOR NEVADA,MIKE DAVIS,3325 NORTH NELLIS,NaN,LAS VEGAS,NV,89115,P,H,REP,T,NaN,NaN,H8NV04067,"DAVIS, MIKE",3325 NORTH NELLIS,"3325 NORTH NELLIS, LAS VEGAS, NV"
9761,C00683573,STRENGTH AND PROGRESS INC,MICHAEL DAVIS,30 N GOULD ST,STE 7981,SHERIDAN,WY,82801,U,N,NaN,Q,NaN,NaN,NaN,"DAVIS, MICHAEL",30 N GOULD ST,"30 N GOULD ST, SHERIDAN, WY"


In [81]:
dones.extend(findall(' davis'))

In [82]:
findall(['phillips', 'ob'])

['ROB PHILLIPS', 'ROBERT PHILLIPS', 'ROBET PHILLIPS']

In [83]:
new = 'ROBERT PHILLIPS' 
dones.append(new)

these = findall(['phillips', 'ob'])

getall(these, new)

orig len: 42
new len: 46


In [84]:
findall(' mele')

['ELOISA MELENDEZ', 'STEVE MELE', 'STEVEN MELE']

In [85]:
cm[cm['tres_nm'] == 'STEVE MELE']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
4342,C00507574,MOTOR CITY PAC,STEVE MELE,611 PENNSYLVANIA AVE SE,STE 143,WASHINGTON,DC,20003,D,Q,NaN,Q,NaN,PETERS,NaN,"MELE, STEVE",611 PENNSYLVANIA AVENUE SE,"611 PENNSYLVANIA AVE SE, WASHINGTON, DC"
7478,C00650366,LESLIE COCKBURN FOR CONGRESS,STEVE MELE,PO BOX 186,NaN,SPERRYVILLE,VA,22740,P,H,DEM,T,NaN,NaN,H8VA05155,"MELE, STEVE",PO BOX 186,"PO BOX 186, SPERRYVILLE, VA"
10603,C00693853,PETERS VICTORY 2020,STEVE MELE,611 PENNSYLVANIA AVE SE,STE. 143,WASHINGTON,DC,20003,J,N,NaN,Q,NaN,NONE,NaN,"MELE, STEVE",611 PENNSYLVANIA AVENUE SE,"611 PENNSYLVANIA AVE SE, WASHINGTON, DC"
14070,C00730390,SARA GIDEON VICTORY FUND,STEVE MELE,611 PENNSYLVANIA AVE SE,NUM 143,WASHINGTON,DC,20003,J,N,NaN,Q,NaN,NONE,NaN,"MELE, STEVE",611 PENNSYLVANIA AVE SE,"611 PENNSYLVANIA AVE SE, WASHINGTON, DC"


In [86]:
cm[cm['tres_nm'] == 'STEVEN MELE'].head()

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
4589,C00525402,RUIZ VICTORY FUND,STEVEN MELE,PO BOX 3433,NaN,PALM DESERT,CA,92261,J,N,NaN,Q,NaN,NONE,H2CA36439,"MELE, STEVEN",PO BOX 3433,"PO BOX 3433, PALM DESERT, CA"
4612,C00526657,TOWARD TOMORROW PAC,STEVEN MELE,PO BOX 544,NaN,SAN ANTONIO,TX,78292,D,Q,NaN,T,NaN,NaN,NaN,"MELE, STEVEN",PO BOX 544,"PO BOX 544, SAN ANTONIO, TX"
5392,C00569871,DOING RIGHT - RESULTS ACTION UNITY LEADERSHIP PAC,STEVEN MELE,PO BOX 3433,NaN,PALM DESERT,CA,92261,D,N,NaN,Q,NaN,RUIZ,NaN,"MELE, STEVEN",PO BOX 3433,"PO BOX 3433, PALM DESERT, CA"
6045,C00606939,ROSEN FOR NEVADA,STEVEN MELE,PO BOX 27195,NaN,LAS VEGAS,NV,89126,P,S,DEM,Q,NaN,ROSEN VICTORY FUND,S8NV00156,"MELE, STEVEN",PO BOX 27195,"PO BOX 27195, LAS VEGAS, NV"
6525,C00626317,GREAT LAKES VICTORY FUND,STEVEN MELE,611 PENNSYLVANIA AVE SE,#143,WASHINGTON,DC,20003,J,N,NaN,Q,NaN,NONE,S4MI00355,"MELE, STEVEN",611 PENNSYLVANIA AVE. SE,"611 PENNSYLVANIA AVE SE, WASHINGTON, DC"


In [87]:
new = 'STEVEN MELE'
dones.append(new)

these = findall(['steve', ' mele'])

getall(these, new)

orig len: 38
new len: 42


In [88]:
findall(['su', 'jackson'])

['SUE JACKSON', 'SUSAN JACKSON']

In [89]:
new = 'SUSAN JACKSON'
dones.append(new)

these = findall(['su', 'jackson'])

getall(these, new)

orig len: 3
new len: 40


In [90]:
findall(' otte')

['BEN OTTENHOFF', 'BENJAMIN OTTENHOFF']

In [91]:
new = 'BENJAMIN OTTENHOFF'
dones.append(new)

these = findall(' otte')

getall(these, new)

orig len: 37
new len: 38


In [92]:
findall(' mcmi')

['ANN MCMILLAN',
 'BENIAH MCMILLER',
 'COLLIN MCMICHAEL',
 'EVA MCMILLAN',
 'GEETA MCMILLAN',
 'KURT MCMILLAN',
 'MARY MCMILLAN']

In [93]:
dones.extend(findall(' mcmi'))

In [94]:
findall(' lew')

['ANDY LEWIS JAMES',
 'BRANDON LEWIS',
 'CATHLEEN RANA LEWIS',
 'CHERYL LEWIS',
 'CORDELIA LEWIS BURKS',
 'DALE LEWIS',
 'DENISE LEWIS',
 'JACK LEWIS CHARBONNEAU',
 'JEFF LEWIS',
 'JENNIFER LEWIS',
 'KATHRYN LEWIS',
 'KEET LEWIS',
 'KRIS LEWIS',
 'LEROY LEWIS',
 'LINDSAY LEWIS',
 'MARK LEWIS',
 'MARY PARKER LEWIS',
 'MATT LEWIS',
 'MICHAEL LEWIS',
 'NATHAN LEWIS WURTZEL',
 'RICHARD LEWIS',
 'RONNIE LEWIS ALSTON',
 'ROSS BAZELON LEWIN',
 "SABRINA Y'VES LEWIS-JONES",
 'SAGE LEWIS',
 'SCOTT LEWIS',
 'TOM LEWELLEN',
 'WENDY LEWIS',
 'WENDY LEWIS ARMSTRONG']

In [95]:
findall(['and', ' james'])

['ANDY LEWIS JAMES', 'BRANDON MICHAEL JAMES', 'CHANDLER JAMES HOUGHTLEN']

In [96]:
findall(['cord', 'burk'])

['CORDELIA LEWIS BURKS']

In [97]:
findall('charbon')

['JACK LEWIS CHARBONNEAU']

In [98]:
findall(['nat', 'wurtz'])

['NATHAN LEWIS WURTZEL']

In [99]:
findall(' alston')

['DONNA JEAN ALSTON', 'LATONYA ALSTON', 'LINDA ALSTON', 'RONNIE LEWIS ALSTON']

In [100]:
cm[cm['tres_nm'] == 'WENDY LEWIS']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
3414,C00450411,MOMENTUM PAC,WENDY LEWIS,1200 SMITH,SUITE 1600,HOUSTON,TX,77002,U,N,NaN,A,NaN,NaN,NaN,WENDY LEWIS,1200 SMITH,"1200 SMITH, HOUSTON, TX"


In [101]:
cm[cm['tres_nm'] == 'WENDY LEWIS ARMSTRONG']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
12368,C00712810,1958 PAC,WENDY LEWIS ARMSTRONG,4008 CHARTRES,STE 100,HOUSTON,TX,77004,U,N,NaN,Q,NaN,NONE,NaN,"LEWIS ARMSTRONG, WENDY L. MRS.",4008 CHARTRES,"4008 CHARTRES, HOUSTON, TX"


In [102]:
new = 'WENDY LEWIS'
dones.append(new)

these = findall('wendy lewis')

getall(these, new)

orig len: 1
new len: 2


In [103]:
dones.extend(findall(' lew'))

In [104]:
findall(' lawl')

['CHRISTOPHER LAWLOR', 'KELLY LAWLER']

In [105]:
dones.extend(findall(' lawl'))

In [106]:
findall(' crum')

['GARY CRUMMITT', 'JACOB MITCHELL CRUM', 'JANE CRUMPLER', 'JOHN CRUMP']

In [107]:
dones.extend(findall(' crum'))

In [108]:
findall(' slater')

['ANNE SLATER', 'JEN SLATER', 'SAMANTHA SLATER', 'TODD SLATER']

In [109]:
dones.extend(findall(' slater'))

In [110]:
findall(' mccaul')

['ALICIA MCCAULEY', 'MIKE MCCAULEY', 'STEVE MCCAULEY']

In [111]:
dones.extend(findall(' mccaul'))

In [112]:
findall(' montg')

['CHRISTOPHER MONTGOMERY WOODFIN',
 'DAVA MONTGOMERY',
 'DIANE MONTGOMERY',
 'JAMAR MONTGOMERY',
 'JAMES MONTGOMERY',
 'MEGAN MONTGOMERY',
 'MIMI MONTGOMERY',
 'RUSSELL MONTGOMERY',
 'THOMAS MONTGOMERY',
 'WILLIAM FLEMING MONTGOMERY']

In [113]:
dones.extend(findall(' montg'))

In [114]:
findall(' nis')

['HARRY NISKA', 'MELISSA NISSEN']

In [115]:
dones.extend(findall(' nis'))

In [116]:
findall(' matthews')

['LAURA MATTHEWS', 'PAUL MATTHEWS']

In [117]:
dones.extend(findall(' matthews'))

In [118]:
findall(' goul')

['ADAM GOULD',
 'ADAM KOHL GOULD',
 'DAVID GOULD',
 'JANICE GOULD',
 'VAN CHARLES GOULD']

In [119]:
cm[cm['tres_nm'].apply(lambda x: ('ADAM' in x) & ('GOULD' in x))]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
10861,C00696807,STEVE LYNE,ADAM GOULD,35 RICE ST,NaN,CAMBRIDGE,MA,02140,P,P,UN,Q,NaN,NONE,P00010330,"GOULD, ADAM",35 RICE STREET,"35 RICE ST, CAMBRIDGE, MA"
10865,C00696849,THE COMMITTEE TO ELECT STEPHEN LYNE,ADAM KOHL GOULD,35 RICE ST,NaN,CAMBRIDGE,MA,02140,A,P,IND,Q,NaN,NaN,P00010330,"GOULD, ADAM KOHL",35 RICE STREET,"35 RICE ST, CAMBRIDGE, MA"


In [120]:
new = 'ADAM KOHL GOULD'
dones.append(new)

these = findall(['adam', 'gould'])

getall(these, new)

orig len: 1
new len: 2


In [121]:
dones.extend(findall(' goul'))

In [122]:
findall(' gan')

['BRIAN GANNON',
 'CHARLES GANTT',
 'GREG GANDRUD',
 'MARIA VICTORIA GANACIAS BORJA',
 'MARY RANDOLPH GANNON',
 'RAGINI GANDHI',
 'RON GANT']

In [123]:
findall('borja')

['MARIA VICTORIA GANACIAS BORJA']

In [124]:
findall(['mary', 'randolph'])

['MARY RANDOLPH GANNON']

In [125]:
dones.extend(findall(' gan'))

In [126]:
findall(' carli')

['JAMES CARLISLE', 'MICHAEL CARLIN', 'ROBERT CARLIN']

In [127]:
dones.extend(findall(' carli'))

In [128]:
findall(' kra')

['ALLEN KRAMER',
 'CATHERINE KRANTZ',
 'CHRISTINE LYNN KRAMAR',
 'CRYSTAL KRASON',
 'DANIEL KRAMER',
 'DAVID KRALLE',
 'DAVID KRAMER',
 'JEREMY KRAJICEK',
 'JOHN KRALL',
 'KERRIE KRAMER',
 'PATRICK KRASON',
 'THOMAS KRAUS',
 'TOM KRAUSE']

In [129]:
cm[cm['tres_nm'] == 'THOMAS KRAUS']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
1343,C00245530,AMERICAN SOCIETY OF HEALTH SYSTEM PHARMACIST -...,THOMAS KRAUS,4500 EAST-WEST HIGHWAY,NaN,BETHESDA,MD,20814,B,Q,NaN,M,M,AMERICAN SOCIETY OF HELATH SYSTEM PHARMACISTS,NaN,"KRAUS, THOMAS",4500 EAST-WEST HIGHWAY SUITE 900,"4500 EAST-WEST HIGHWAY, BETHESDA, MD"


In [130]:
cm[cm['tres_nm'] == 'TOM KRAUSE']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
9521,C00680504,BROADCOM INC. POLITICAL ACTION COMMITTEE (BROA...,TOM KRAUSE,1320 RIDDER PARK DR,NaN,SAN JOSE,CA,95131,U,Q,NaN,M,C,NaN,NaN,"KRAUSE, TOM",1320 RIDDER PARK DR.,"1320 RIDDER PARK DR, SAN JOSE, CA"


In [131]:
dones.extend(findall(' kra'))

In [132]:
findall(' koch')

['CATHIE KOCH',
 'KEVIN KOCH',
 'LINDA KOCH',
 'NICHOLAS KOCH',
 'THEODORE KOCH',
 'TIMOTHY KOCH',
 'TOM KOCHAN']

In [133]:
dones.extend(findall(' koch'))

In [134]:
findall(' win')

['ADAM WINSTON',
 'ALISON WINGATE',
 'AMY LYNN WINCHESTER',
 'ANDREW WINTERING',
 'ANTWAIN WINTERS',
 'ARTHUR WINSTEAD',
 'BENJAMIN VAN WINKLE',
 'BRIGEN WINTERS',
 'CHARLIE MADISON WINTERS',
 'DEAN WINE',
 'DONNA FOLEY WINTERSON',
 'ETHAN WINGFIELD',
 'GERALD WINDHAM',
 'KAREN WINGARD',
 'MARC WINGER',
 'MIKE WINDER',
 'PATRICIA WINTER',
 'SCOTT WINKLER',
 'STEVE WIND',
 'THOMAS FRANCIS WINTERBOTTOM',
 'THOMAS WINTER',
 'TRAVIS WINES',
 'VICKIE WINPISINGER',
 'VICTORIA WINTRODE']

In [135]:
dones.extend(findall(' win'))

In [136]:
findall(' rit')

['ANDREW RITTENBERG',
 'JENNIFER RITCHIE',
 'JOEL RITER',
 'KENNETH RITER',
 'RUTH RITCHEY']

In [137]:
dones.extend(findall(' rit'))

In [138]:
findall(' tat')

['ALEXANDRIA TATE',
 'BRIAN TATUM',
 'BRYAN TATUM',
 'DARRYL TATTRIE',
 'JOHN TATE',
 'MARTHA SHAW TATE',
 'RYAN TATE',
 'SAMBA TATA',
 'SRINIVASA TATINENI']

In [139]:
cm[cm['tres_nm'].apply(lambda x: x in ['BRIAN TATUM', 'BRYAN TATUM'])]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
2896,C00415026,AMERICAN FUELS AND PETROCHEMICAL MANUFACTURERS...,BRIAN TATUM,1800 M ST NW,SUITE 900 NORTH,WASHINGTON,DC,20036,B,Q,NaN,M,T,AMERICAN FUELS AND PETROCHEMICAL MANUFACTURERS...,NaN,"TATUM, BRIAN",1800 M STREET NW,"1800 M ST NW, WASHINGTON, DC"
13450,C00723940,TATUM FOR CONGRESS,BRYAN TATUM,12485 NW HASKELL CT,NaN,PORTLAND,OR,97229,P,H,REP,Q,NaN,NONE,H0OR01178,"TATUM, BRYAN",12485 NW HASKELL CT,"12485 NW HASKELL CT, PORTLAND, OR"


In [140]:
dones.extend(findall(' tat'))

In [141]:
findall('ralls')

['STEVE RALLS', 'STEVEN RALLS']

In [142]:
cm[cm['tres_nm'] == 'STEVEN RALLS']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
2588,C00395467,JEFF FORTENBERRY FOR UNITED STATES CONGRESS,STEVEN RALLS,PO BOX 30265,NaN,LINCOLN,NE,68503,P,H,REP,Q,NaN,FORTENBERRY-SCOTT VICTORY FUND,H4NE01064,"RALLS, STEVEN",PO BOX 30265,"PO BOX 30265, LINCOLN, NE"


In [143]:
cm[cm['tres_nm'] == 'STEVE RALLS'].head()

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
3051,C00426494,FOR OUR REPUBLIC'S TRADITIONS FUND AKA FORT FUND,STEVE RALLS,PO BOX 30883,NaN,LINCOLN,NE,68503,D,N,NaN,M,NaN,JEFF FORTENBERRY,NaN,"RALLS, STEVE MR.",PO BOX 30883,"PO BOX 30883, LINCOLN, NE"
3070,C00427781,ONLINE LENDERS ALLIANCE POLITICAL ACTION COMMI...,STEVE RALLS,PO BOX 15480,SE STATION,WASHINGTON,DC,20003,U,Q,NaN,M,M,ONLINE LENDERS ALLIANCE,NaN,"RALLS, STEVE",PO BOX 15480,"PO BOX 15480, WASHINGTON, DC"
3279,C00440115,STEVE AUSTRIA FOR CONGRESS,STEVE RALLS,2537 OBETZ DR,NaN,BEAVERCREEK,OH,45434,P,H,REP,Q,NaN,NONE,H8OH07065,"RALLS, STEVE",2537 OBETZ DR,"2537 OBETZ DR, BEAVERCREEK, OH"
4193,C00498345,MULLIN FOR CONGRESS,STEVE RALLS,PO BOX 3681,NaN,MUSKOGEE,OK,74402,P,H,REP,Q,NaN,MULLIN VICTORY FUND,H2OK02083,"RALLS, STEVE",PO BOX 3681,"PO BOX 3681, MUSKOGEE, OK"
4930,C00547976,"BEN SASSE FOR U.S. SENATE, INC.",STEVE RALLS,700 R ST,UNIT 83978,LINCOLN,NE,685010349,P,S,REP,Q,NaN,SASSE LEADERSHIP COMMITTEE,S4NE00090,"RALLS, STEVE",700 R ST,"700 R ST, LINCOLN, NE"


In [144]:
new = 'STEVE RALLS'
dones.append(new)

these = findall('ralls')

getall(these, new)

orig len: 19
new len: 20


In [145]:
findall(' has')

['BECKY HASSLEN',
 'CHRISSIE HASTIE',
 'GARY HASTY',
 'JULIA HASTON',
 'MUAD HASSAN',
 'VECTOR HASTING']

In [146]:
dones.extend(findall(' has'))

In [147]:
findall(' buch')

['EVAN BUCHHOLZ',
 'KATHERINE BUCHANAN',
 'KIMBERLY BUCHAN',
 'MARY ANNE BUCHANAN',
 'PATRICIA BUCHANAN',
 'SHAWN BUCHTEL',
 'SUSAN MAUDE BUCHSER']

In [148]:
dones.extend(findall(' buch'))

In [149]:
findall(' bac')

['ANDREW BACON',
 'CATHARYNE BACH',
 'DAN BACHUS',
 'DAN BACKER',
 'DANIEL BACINE',
 'JA BACHMAN',
 'JASON BACON',
 'JOE BACA',
 'JOEL BACON',
 'JOSEPH BACHMAN',
 'MARLA BACHUS',
 'RJ BACON',
 'SHAWN BACHAR']

In [150]:
dones.extend(findall(' bac'))

In [151]:
findall(' brog')

['KEVIN BROGHAMER', 'LEE ANNE BROGOWSKI', 'MICHAEL BROGAN']

In [152]:
dones.extend(findall(' brog'))

In [153]:
findall(' gia')

['ANDREW GIANNONE',
 'FRANCIS GIARDIELLO',
 'HOLLY GIARRAPUTO',
 'JACOB GIARDINA',
 'JOEL GIANNELLI',
 'LISA ANNE GIANUKOS',
 'SUSAN GIANNETTI LONGACRE']

In [154]:
findall('longacre')

['SUSAN GIANNETTI LONGACRE']

In [155]:
dones.extend(findall(' gia'))

In [156]:
findall(' deane')

['SHAWNDA DEANE']

In [157]:
dones.extend(findall(' deane'))

In [158]:
findall(' hag')

['CHADWICK HAGAN',
 'HILLARY HAGERTY',
 'JANICE GWYN HAGERMAN',
 'JOHN HAGY',
 'LORA HAGGARD',
 'PAUL HAGY',
 'SARAH HAGER',
 'SEAN HAGGARD',
 'TIMOTHY HAGAN']

In [159]:
dones.extend(findall(' hag'))

In [160]:
findall(' anger')

['LINDSAY ANGERHOLZER']

In [161]:
dones.extend(findall(' anger'))

In [162]:
findall(' lowe')

['AARON LOWE',
 'DONALD EUGENE LOWE',
 'GRETCHEN LOWE',
 'JENNIFER LOWE',
 'KEITH LOWEY',
 'MATTHEW LOWE',
 'PATRICK LOWERY',
 'ROSS LOWE',
 'SANDRA LOWE',
 'TRAVIS LOWE']

In [163]:
dones.extend(findall(' lowe'))

In [164]:
findall(' bau')

['AMBER BAUR',
 'BRIAN BAUER',
 'CARL BAUMAN',
 'CONNIE BAUER',
 'DAVID BAUER',
 'DAWN BAUMAN',
 'GEOFFREY BAUMAN',
 'GERALD LYNN BAUER',
 'ISAAC BAUER',
 'JAMIE BAUMANN',
 'JULIAN BAUTISTA MARTINEZ',
 'KEITH BAUER',
 'NATALIE BAUR',
 'PAULA BAUER',
 'ROBERT BAUTER',
 'TED BAUER',
 'TIMOTHY RYAN BAUER']

In [165]:
findall(['ju', 'mart'])

['JULIAN BAUTISTA MARTINEZ']

In [166]:
dones.extend(findall(' bau'))

In [167]:
findall(' evans')

['APRIL EVANS DOYLE',
 'BRENT EVANS',
 'BRIAN EVANS',
 'DIANE EVANS',
 'DOUG EVANS',
 'GEORGE EVANS',
 'JOHN EVANS',
 'JONATHAN EVANS',
 'LYLE EVANS',
 'MARTA BLEA EVANS',
 'MELISSA ANNE EVANS',
 'MIKE EVANS',
 'NICHOLAS EVANS',
 'NICK EVANS',
 'SUE EVANS',
 'WALTER EVANS']

In [168]:
findall(' doyle')

['APRIL EVANS DOYLE',
 'JACOB DOYLE',
 'JOHN DOYLE',
 'MATTHEW DOYLE',
 'TIM DOYLE']

In [169]:
dones.extend(findall(' doyle'))

In [170]:
dones.extend(findall(' evans'))

In [171]:
findall(' mason')

['DAVE MASON',
 'DAVID MASON',
 'DAVID MASON SISK',
 'DIRK MASON CANTRELL',
 'DYANA MASON',
 'JAMES GRAY MASON',
 'JOHN LEONARD MASON',
 'LINDA MASON',
 'MARCUS MASON',
 'MICHAEL MASON',
 'MICHAEL THOMAS MASON',
 'NICHOLAS LEONARD MASON',
 'PAUL DOUGLAS MASON HALE',
 'STACY MASON']

In [172]:
cm[cm['tres_nm'] == 'DAVE MASON'].head()

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
2737,C00406850,BLUE CROSS BLUE SHIELD OF SOUTH CAROLINA FEDER...,DAVE MASON,INTERSTATE 20 AT ALPINE RD,NaN,COLUMBIA,SC,29214,B,Q,NaN,Q,C,NaN,NaN,"MASON, DAVE MR.",INTERSTATE 20 AT ALPINE ROAD,"INTERSTATE 20 AT ALPINE RD, COLUMBIA, SC"


In [173]:
cm[(cm['tres_nm'] == 'DAVID MASON') & (cm['cmte_st'] == 'SC')].head()

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address


In [174]:
findall('cantrell')

['DIRK MASON CANTRELL', 'MIKE CANTRELL']

In [175]:
cm[cm['tres_nm'] == 'MICHAEL THOMAS MASON'].head()

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
6907,C00635268,ROBERT4REP,MICHAEL THOMAS MASON,PO BOX 867614,NaN,PLANO,TX,75086,P,H,IND,T,NaN,NaN,H8TX03131,"MASON, MICHAEL THOMAS",PO BOX 867614,"PO BOX 867614, PLANO, TX"


In [176]:
cm[(cm['tres_nm'] == 'MICHAEL MASON') & (cm['cmte_st'] == 'TX')].head()

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
1475,C00268904,TRINITY INDUSTRIES EMPLOYEE POLITICAL ACTION C...,MICHAEL MASON,2525 N STEMMONS FREEWAY,NaN,DALLAS,TX,75207,B,Q,NaN,M,C,TRINITY INDUSTRIES INC.,NaN,"MASON, MICHAEL J.",2525 N. STEMMONS FREEWAY,"2525 N STEMMONS FREEWAY, DALLAS, TX"


In [177]:
new = 'MICHAEL MASON'
dones.append(new)

these = findall(['michael', 'mason'])

getall(these, new)

orig len: 1
new len: 2


In [178]:
findall(' hale')

['CHRISTINA HALEY',
 'DAKOTA HALE',
 'DONALD BRETT HALE',
 'JAMES HALE',
 'JANELL HALES',
 'KAREN HALE',
 'LINDA HALE',
 'NANCY HALEY',
 'PAUL DOUGLAS MASON HALE',
 'RANDY HALE',
 'RYAN HALE',
 'SANDRA MARIE HALE',
 'TONY HALE']

In [179]:
dones.extend(findall(' mason'))

In [180]:
findall(' lloyd')

['CHASE LLOYD',
 'JOHN LLOYD',
 "JUNE LA'GAY LLOYD",
 'JUSTIN LLOYD MCNEEL',
 'PHILIP LLOYD',
 'WILLIS LLOYD']

In [181]:
findall(' mcneel')

['JUSTIN LLOYD MCNEEL']

In [182]:
dones.extend(findall(' lloyd'))

In [183]:
findall(' johnson')

['ALEXIS JOHNSON',
 'ANDREW JOHNSON',
 'AQUASIA JOHNSON',
 'ARLEN JOHNSON',
 'ARNOLD JOHNSON',
 'BETTY JOHNSON',
 'CAMILLE JOHNSON',
 'CATHERINE JOHNSON',
 'CHARLES JOHNSON',
 'CHELSEA JOHNSON',
 'CHRIS JOHNSON',
 'CHRISTOPHER LEE JOHNSON',
 'CHRYSALIS JOHNSON',
 'CORY JOHNSON',
 'CRYSTAL JOHNSON',
 'DARRYL JOHNSON',
 'DAVID JOHNSON',
 'DEBORAH ANNE JOHNSON',
 'DIANE JOHNSON',
 'DOUGLAS JOHNSON',
 'DUANE JOSEPH JOHNSON',
 'EDGAR JOHNSON',
 'ERIK ARLEN JOHNSON',
 'GEORGE JOHNSON',
 'GREGORY JOHNSON',
 'JAMES JOHNSON',
 'JANET JOHNSON',
 'JAY JOHNSON',
 'JERRY JOHNSON',
 'JESSICA FURST JOHNSON',
 'JIM JOHNSON',
 'JOHN JOHNSON',
 'JOSEPH JOHNSON',
 'JUDITH JOHNSON',
 'JUSTINE JOHNSON',
 'KENNETH JOHNSON',
 'LINELL JOHNSON',
 'LORETTA JOHNSON',
 'LORRETTA JOHNSON',
 'MALISSA JOHNSON',
 'MARK JOHNSON',
 'MARY JOHNSON',
 'MATTHEW JOHNSON',
 'MELANIE ELIZABETH JOHNSON',
 'MELODIE JOHNSON',
 'MICHAEL JOHNSON',
 'MIRIAM JOHNSON GIBSON',
 'OTIS LEE JOHNSON',
 'REBECCA JOHNSON',
 'REGINALD JOHNSO

In [184]:
cm[cm['tres_nm'] == 'ARLEN JOHNSON']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
8024,C00659227,COMMITTEE TO ELECT KEVIN POWERS,ARLEN JOHNSON,4 RONDEN RD,NaN,MILLVILLE,MA,01529,P,H,REP,Q,NaN,NONE,H8MA02090,"JOHNSON, R. ARLEN",4 RONDEN ROAD,"4 RONDEN RD, MILLVILLE, MA"


In [185]:
cm[cm['tres_nm'] == 'ERIK ARLEN JOHNSON']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
9963,C00686238,RESISTERY PAC,ERIK ARLEN JOHNSON,275 LAKE DR,NaN,KENSINGTON,CA,94708,U,O,NaN,Q,NaN,NaN,NaN,ERIK ARLEN JOHNSON,275 LAKE DRIVE,"275 LAKE DR, KENSINGTON , CA"


In [186]:
cm[cm['tres_nm'] == 'CHRIS JOHNSON']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
4628,C00528166,NATIONAL ASSOCIATION FOR FIXED ANNUITIES POLIT...,CHRIS JOHNSON,1155 F ST NW,SUITE 1050,WASHINGTON,DC,20004,U,Q,NaN,Q,T,NaN,NaN,"JOHNSON, CHRIS",1155 F ST NW,"1155 F ST NW, WASHINGTON, DC"


In [187]:
cm[cm['tres_nm'] == 'CHRISTOPHER LEE JOHNSON']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
11655,C00705418,CHRISTOPHER LEE JOHNSON FOR PRESIDENT FOR AN I...,CHRISTOPHER LEE JOHNSON,203 DOVIE DR,NaN,GREER,SC,29651,P,P,REP,Q,NaN,NONE,P00011890,"JOHNSON, CHRISTOPHER LEE --SELECT A LEE",203 DOVIE DRIVE,"203 DOVIE DR, GREER, SC"


In [188]:
cm[cm['tres_nm'] == 'JIM JOHNSON']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
4244,C00501429,TD BANK N.A. POLITICAL ACTION COMMITTEE,JIM JOHNSON,125 PARK AVE 24TH FLOOR,NaN,NEW YORK,NY,10017,B,Q,NaN,Q,C,TD BANK N.A.,NaN,"JOHNSON, JIM",125 PARK AVENUE 24TH FLOOR,"125 PARK AVE 24TH FLOOR, NEW YORK, NY"


In [189]:
cm[cm['tres_nm'] == 'JAMES JOHNSON']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
1294,C00236034,WERNER ENTERPRISES INC POLITICAL ACTION COMMITTEE,JAMES JOHNSON,14507 FRONTIER RD,NaN,OMAHA,NE,68138,U,Q,NaN,Q,C,WERNER ENTERPRISES INC,NaN,"JOHNSON, JAMES L.",14507 FRONTIER ROAD,"14507 FRONTIER RD, OMAHA, NE"
2860,C00412569,DENTON COUNTY REPUBLICAN VICTORY FUND,JAMES JOHNSON,2921 COUNTRY CLUB RD,NaN,DENTON,TX,76202,U,X,REP,M,M,REPUBLICAN PARTY OF TEXAS,NaN,"JOHNSON, JAMES S.",2921 COUNTRY CLUB RD #102,"2921 COUNTRY CLUB RD, DENTON, TX"


In [190]:
cm[cm['tres_nm'] == 'LORETTA JOHNSON']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
14817,C70002472,"AMERICAN FEDERATION OF TEACHERS, AFL-CIO",LORETTA JOHNSON,555 NEW JERSEY AVE NW,NaN,WASHINGTON,DC,20001,U,C,NaN,Q,L,NaN,NaN,"JOHNSON, LORETTA","555 NEW JERSEY AVENUE, N.W.","555 NEW JERSEY AVE NW, WASHINGTON, DC"


In [191]:
cm[cm['tres_nm'] == 'LORRETTA JOHNSON']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
210,C00028860,"AMERICAN FEDERATION OF TEACHERS, AFL-CIO COMMI...",LORRETTA JOHNSON,555 NEW JERSEY AVE NW,NaN,WASHINGTON,DC,20001,B,Q,NaN,Q,L,NaN,NaN,"JOHNSON, LORRETTA","555 NEW JERSEY AVENUE, NW","555 NEW JERSEY AVE NW, WASHINGTON, DC"


In [192]:
new = 'LORRETTA JOHNSON'
dones.append(new)

these = findall(['lor', 'etta', 'johnson'])

getall(these, new)

orig len: 1
new len: 2


In [193]:
findall(['mi', 'gibson'])

['MICHAEL GIBSON', 'MIRIAM JOHNSON GIBSON']

In [194]:
dones.extend(findall(' johnson'))

In [195]:
findall(' goldstein')

['JULIAN GOLDSTEIN', 'STEPHEN GOLDSTEIN']

In [196]:
dones.extend(findall(' goldstein'))

In [197]:
findall(' marks')

['BRUCE MARKS',
 'DOUG MARKS',
 'MICHELLE MARKS-OSBORNE',
 'NANCY MARKS',
 'STEPHEN MARKS']

In [198]:
dones.extend(findall(' marks'))

In [199]:
findall(' grav')

['RONALD GRAVINO', 'SCOTT GRAVES', 'SHARRON GRAVES', 'STEPHANIE GRAVES']

In [200]:
dones.extend(findall(' grav'))

In [201]:
findall(' eich')

['ALYSSA EICHMAN', 'RICHARD EICHMAN']

In [202]:
dones.extend(findall(' eich'))

In [203]:
findall(' fou')

['BRIAN FOUCART', 'HERMAN FOUSHEE', 'JOSEPH FOUNTAIN', 'ROGER FOUNTAIN']

In [204]:
dones.extend(findall(' fou'))

In [205]:
findall(' martin')

['ANDY MARTIN',
 'ANTONIO MARTINEZ',
 'CASSANDRA MARTINEZ',
 'CECILIA MARTINEZ SALAZAR',
 'CHARLES MARTIN',
 'CHRIS MARTIN',
 'DAVID MARTIN',
 'DONALD MARTIN',
 'ED MARTIN',
 'ERNESTO MARTINEZ',
 'GAYLE MARTIN',
 'GEMMA MARTIN',
 'JANE MARTIN',
 'JANICE MARTIN',
 'JEAN MARTINEZ',
 'JENA TONICE MARTIN',
 'JENNIFER MARTIN',
 'JOHN MARTIN',
 'JOHN PATRICK MARTINI',
 'JONATHAN MARTIN',
 'JOSEPH MARTIN',
 'JOSH MARTIN',
 'JULIAN BAUTISTA MARTINEZ',
 'KARRIE MARTIN',
 'KEVIN MARTIN PRCIN SALZBURG-FELTS',
 'LARISSA MARTINEZ',
 'LAWRENCE MARTIN',
 'LILIANA MARTINEZ',
 'LISA MARTINEZ',
 'LOUIE CRUZ MARTINEZ',
 'MARIO MARTINEZ',
 'MARNEY MARTIN',
 'MATTHEW MARTIN',
 'MAURA MARTINEZ',
 'NOVEL MARTIN',
 'PAUL MARTINO',
 'ROY MARTINEZ',
 'SARAH MARTIN',
 'SELENA ROSE MARTINEZ',
 'SHARI MARTIN',
 'STEVE MARTIN',
 'STEVEN MARTIN',
 'SUSAN MARTIN',
 'SUSANANN MARTIN',
 'THOMAS MARTIN',
 'TODD MARTIN',
 'URIEL MARTINEZ',
 'WILLIAM MARTIN',
 'XAVIER MARTINEZ']

In [206]:
findall(' salazar')

['ANNA SALAZAR', 'CECILIA MARTINEZ SALAZAR', 'DEIAN SALAZAR']

In [207]:
findall('salzburg')

['KEVIN MARTIN PRCIN SALZBURG-FELTS']

In [208]:
cm[cm['tres_nm'] == 'STEVE MARTIN']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
14289,C00732685,BURRIL FOR CONGRESS,STEVE MARTIN,PO BOX 1039,NaN,SKYLAND,NC,28776,P,H,REP,Q,NaN,NONE,H0NC11217,"MARTIN, STEVE",P.O. BOX 1039,"PO BOX 1039, SKYLAND, NC"


In [209]:
cm[(cm['tres_nm'] == 'STEVEN MARTIN') & (cm['cmte_st'] == 'NC')]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address


In [210]:
dones.extend(findall(' martin'))

In [211]:
findall(' davidson')

['ALLEN DAVIDSON',
 'CARY DAVIDSON',
 'DWIGHT DAVIDSON',
 'NATHAN DAVIDSON',
 'PAUL DAVIDSON REYNOLDS',
 'REBECCA DAVIDSON',
 'ROBERT DAVIDSON']

In [212]:
findall(['paul', 'reynolds'])

['PAUL DAVIDSON REYNOLDS', 'PAUL REYNOLDS']

In [213]:
cm[cm['tres_nm'] == 'PAUL REYNOLDS']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
7429,C00649459,DIANE FOR COLORADO CD3,PAUL REYNOLDS,PO BOX 771606,NaN,STEAMBOAT SPRINGS,CO,80477,P,H,DEM,T,NaN,NaN,H8CO03192,"REYNOLDS, PAUL D. MR.",PO BOX 771606,"PO BOX 771606, STEAMBOAT SPRINGS, CO"
10078,C00687715,DIANE MITSCH BUSH VICTORY FUND,PAUL REYNOLDS,PO BOX 771606,NaN,STEAMBOAT SPRINGS,DC,80477,J,N,NaN,T,NaN,NaN,NaN,"REYNOLDS, PAUL D. MR.",PO BOX 771606,"PO BOX 771606, STEAMBOAT SPRINGS, DC"


In [214]:
cm[cm['tres_nm'] == 'PAUL DAVIDSON REYNOLDS']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
11870,C00707638,DIANE FOR COLORADO CD3,PAUL DAVIDSON REYNOLDS,PO BOX 771606,NaN,STEAMBOAT SPRINGS,CO,80477,P,H,DEM,Q,NaN,NONE,H8CO03192,"REYNOLDS, PAUL DAVIDSON MR.",PO BOX 771606,"PO BOX 771606, STEAMBOAT SPRINGS, CO"


In [215]:
new = 'PAUL DAVIDSON REYNOLDS'
dones.append(new)

these = findall(['paul', 'reynolds'])

getall(these, new)

orig len: 1
new len: 3


In [216]:
dones.extend(findall(' davidson'))

In [217]:
findall(' bren')

['ANNE BRENSLEY',
 'MEGAN BRENGARTH',
 'NICHOLAS BRENT MANTANONA',
 'OZY BRENNNAN']

In [218]:
findall(' mantan')

['NICHOLAS BRENT MANTANONA']

In [219]:
dones.extend(findall(' bren'))

In [220]:
findall(' mont')

['CARROLL MONTAGUE MS/MR PRICE',
 'CHRISTOPHER MONTGOMERY WOODFIN',
 'DACEY MONTOYA',
 'DAVA MONTGOMERY',
 'DIANE MONTGOMERY',
 'JAMAR MONTGOMERY',
 'JAMES MONTGOMERY',
 'KENNETH JAMES MONTALVO',
 'MARIA MONTANO',
 'MEGAN MONTGOMERY',
 'MIMI MONTGOMERY',
 'PATRICIA MONTAGUE',
 'RUSSELL MONTGOMERY',
 'THOMAS MONTGOMERY',
 'WILLIAM FLEMING MONTGOMERY']

In [221]:
dones.extend(findall(' mont'))

In [222]:
findall(' hank')

['BRENDA HANKINS']

In [223]:
dones.extend(findall(' hank'))

In [224]:
findall(' lesh')

['GREGORY LESHOCK', 'MEREDITH LESHER', 'WARREN LESHNER']

In [225]:
dones.extend(findall(' lesh'))

In [226]:
findall(' bre')

['ANNE BRENSLEY',
 'BAILEY BRESSLER',
 'BEN BREWSTER',
 'BILLY JOE BREWSTER',
 'CARL BREWER',
 'CHRISTOPHER SCOTT BREWER',
 'DONALD BRETT HALE',
 'JAMES BREWER',
 'JAMES TROY BREWER',
 'JOANN BREIVOGEL',
 'JONATHAN BRETT RINGHAM',
 'MEGAN BRENGARTH',
 'NATALIE BREEN',
 'NICHOLAS BRENT MANTANONA',
 'NICOLE BREWIN',
 'OZY BRENNNAN',
 'PATRICIA BREED',
 'PAUL BREAZEALE',
 'TROY BREWER',
 'VICTOR BREED',
 'WADE BREWER']

In [227]:
cm[cm['tres_nm'] == 'JAMES BREWER']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
14710,C00736959,JOHN BARGE FOR CONGRESS,JAMES BREWER,PO BOX 572,NaN,ROME,GA,30162,P,H,REP,Q,NaN,NONE,H0GA14089,"BREWER, JAMES T",PO BOX 572,"PO BOX 572, ROME, GA"


In [228]:
cm[cm['tres_nm'] == 'JAMES TROY BREWER']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
6293,C00617142,STACY RIES SNYDER FOR CONGRESS,JAMES TROY BREWER,5500 NEVIL POINTE,NaN,BRENTWOOD,TN,37027,P,H,REP,T,NaN,NaN,H6TN05322,JAMES TROY BREWER,5500 NEVIL POINTE,"5500 NEVIL POINTE, BRENTWOOD, TN"
9686,C00682633,THE NEW AMERICAN POPULIST PAC,JAMES TROY BREWER,95 WHITE BRIDGE RD,SUITE 207,NASHVILLE,TN,37205,U,O,NaN,Q,NaN,NONE,NaN,"BREWER, JAMES TROY",95 WHITE BRIDGE RD,"95 WHITE BRIDGE RD, NASHVILLE, TN"
11276,C00701177,MERRITT FOR CONGRESS,JAMES TROY BREWER,PO BOX 30501,479 JOHNNY MERCER BLVD,SAVANNAH,GA,31410,P,H,REP,Q,NaN,NONE,H0GA01060,"BREWER, JAMES TROY",PO BOX 30501,"PO BOX 30501, SAVANNAH, GA"
11579,C00704411,VOLUNTEER ISSUES PAC INC,JAMES TROY BREWER,95 WHITE BRIDGE RD,NaN,NASHVILLE,TN,37205,D,N,NaN,Q,NaN,TIM BURCHETT,NaN,JAMES TROY BREWER,95 WHITE BRIDGE RD SUITE 207,"95 WHITE BRIDGE RD, NASHVILLE, TN"


In [229]:
new = 'JAMES BREWER'
dones.append(new)

these = findall(['james', 'brewer'])

getall(these, new)

orig len: 1
new len: 5


In [230]:
findall(' ringham')

['JONATHAN BRETT RINGHAM']

In [231]:
dones.extend(findall(' bre'))

In [232]:
findall(' sel')

['GREGORY SELTZER',
 'JACOB SCOTT SELECT PETERS',
 'JANET SELWAY',
 'JOHN SELPH',
 'KATHY SELVAGGIO',
 'LAURA SELKEN',
 'LAURIE SELF',
 'LINDA SELL',
 'MONIKA SELMONT',
 'PAUL SELTMAN',
 'RODERICK SELLERS',
 'SARAH SELENE SCOTT-STEWART',
 'STEVE SELTZER',
 'THERESA SELL']

In [233]:
findall(['ja', 'peters'])

['JACOB SCOTT SELECT PETERS', 'JAMES WALTER PETERSON']

In [234]:
findall(['sar', 'scot'])

['SARAH SELENE SCOTT-STEWART']

In [235]:
dones.extend(findall(' sel'))

In [236]:
findall(' owen')

['ALEXANDRA OWENSBY',
 'DEBORAH OWENS',
 'GAIL DENISE OWENS',
 'JOHN OWENS',
 'JUNE OWENS',
 'KYLE LEE MASTER OWENS',
 'LINDA OWENS',
 'MICHAEL OWENS',
 'ROBERT OWENS',
 'RODNEY OWEN MOSIER',
 'SARAH OWEN',
 'SELENA OWENS',
 'STACY OWENS',
 'THADDEUS OWENS',
 'WENDI OWEN',
 'WILLIAM LAIRD OWEN',
 'WILLIAM TIMOTHY OWEN']

In [237]:
findall(' mosier')

['RODNEY OWEN MOSIER']

In [238]:
dones.extend(findall(' owen'))

In [239]:
findall(' vo')

['AARON VOSBURGH',
 'CHRISTOPHER VOROS',
 'FRANK VOYACK',
 'JEN GILBERT VOSS',
 'MARILYN VORNKAHL',
 'MARK VOGEL',
 'MARTIN VOGT',
 'MICHELLE BUZZ360 SCHNEEGANS VOLK',
 'PATRICIA VODVARKA',
 'PATRICK VOSS',
 'PAUL VORNHOLT',
 'PETER VOLOSIN',
 'RADOMIR VOJTECH LUZA',
 'RICHARD VOLIVA',
 'ROBERT VOLTMANN',
 'ROGER VON TING',
 'RONALD VOGEL',
 'TANNER VOGT',
 'TARYN VOGEL']

In [240]:
findall(' luza')

['RADOMIR VOJTECH LUZA']

In [241]:
dones.extend(findall(' vo'))

In [242]:
findall(' bol')

['ANGELA BOLTON',
 'APRIL BOLING',
 'CHRISTOPHER BOLLMAN',
 'DANIEL BOLLNER',
 'DANNIE BOLDEN',
 'DIANE BOLAK',
 'DONALD PATRICK BOLENA',
 'JEFFERY BOLTON',
 'KIMBERLY BOLIN',
 'LARRY LEE SCOTT BOLINGER',
 'MICHAEL BOLAND',
 'MICHAEL BOLLENTIN',
 'MICHAEL BOLTON',
 'MIKE BOLAND',
 'NATHAN BOLT',
 'PRENTISS BOLIN',
 'RANDALL SCOTT BOLL',
 'REGINALD BOLDING',
 'SUSAN BOLHAFNER',
 'TANI MARIE BOLTHOUSE',
 'TOM BOLOVINOS']

In [243]:
dones.extend(findall(' bol'))

In [244]:
findall(' hub')

['AARON HUBBARD',
 'ALEXANDER HUBBARD',
 'CHARLES HUBERT HAUSER',
 'EDWARD HUBBARD',
 'MALVA HUBBARD',
 'MARY ELIZABETH HUBBARD',
 'REBECCA LEIGH HUBBARD',
 'SCOTT HUBAY',
 'STEVEN ERICH HUBBARD',
 'TSHOMBE HUBBARD']

In [245]:
findall(' hauser')

['CHARLES HUBERT HAUSER']

In [246]:
dones.extend(findall(' hub'))

In [247]:
findall(' oz')

['NINA OZLU TUNCELI', 'WILLIAM OZANUS']

In [248]:
findall(' tunc')

['NINA OZLU TUNCELI']

In [249]:
dones.extend(findall(' oz'))

In [250]:
findall(' groe')

['ERIC GROEN', 'GIDGET KAY GROENDYK', 'REBECCA GROEN']

In [251]:
dones.extend(findall(' groe'))

In [252]:
findall(' sola')

['DEREK SOLAR',
 'EDEN SOLAR PATTERSON',
 'ISABEL SOLANO',
 'KRISTIN SOLANDER',
 'REY SOLANO']

In [253]:
findall(['ed', 'patterson'])

['EDEN SOLAR PATTERSON']

In [254]:
dones.extend(findall(' sola'))

In [255]:
findall(' rupp')

['DAVID RUPPENICKER', 'JEFFREY RUPPERT', 'LORI RUPPEN']

In [256]:
dones.extend(findall(' rupp'))

In [257]:
findall(' schi')

['ANTHONY SCHIWEIER',
 'DEIRDRE SCHIFELING',
 'ERIC SCHIPPERS',
 'JAMES SCHISSER',
 'JOHN SCHIESS',
 'JOSEPH SCHINTZ',
 'LOUIS SCHIAZZA',
 'MATTHEW SCHIEBLER',
 'NICHOLAS ADAM SCHILLER',
 'SARAH SCHIMDT']

In [258]:
dones.extend(findall(' schi'))

In [259]:
findall(' guin')

['DELTER GUIN', 'GREGORY GUIN', 'LUCINDA GUINN']

In [260]:
dones.extend(findall(' guin'))

In [261]:
findall(' stone')

['ANN STONE',
 'ERIC STONEHAM',
 'KIEL RICHARD STONE',
 'KRISTEN STONER',
 'LISA STONE',
 "O'LENE STONE",
 'RUTH ROCHELLE STONER',
 'SAMUEL STONE',
 'SHERRI STONE',
 'WILLIAM STONE']

In [262]:
dones.extend(findall(' stone'))

In [263]:
findall(' cros')

['CALEB CROSBY',
 'JOSEPH CROSBY',
 'KENNETH CROSS',
 'MARK CROSS',
 'MONDELL CROSBY',
 'SHAWN VINCENT CROSS']

In [264]:
dones.extend(findall(' cros'))

In [265]:
findall(' gla')

['AEIRAMIQUE GLASS',
 'ANGELA GLASS',
 'EZRA GLASER',
 'JENNIFER GLAIM',
 'KAYLA GLAZE',
 'NATHANIEL GLASGOW',
 'PATRICK GLASGOW',
 'TAYLOR GLADDEN HEMMIG',
 'TIM GLADEAU',
 'WANDA GLAUD',
 'WILLIAM GLASS']

In [266]:
findall('hemmig')

['TAYLOR GLADDEN HEMMIG']

In [267]:
dones.extend(findall(' gla'))

In [268]:
findall(' ragan')

['ASHLEY RAGAN', 'JACKI RAGAN', 'JENNIFER RAGAN', 'VIRGINIA RAGAN']

In [269]:
dones.extend(findall(' ragan'))

In [270]:
findall(' mau')

['BOB MAURER',
 'DEWIGHT MAURICE LITTLEJOHN',
 'LES MAUL',
 'NICK MAURO',
 'STACEY MAUD',
 'SUSAN MAUDE BUCHSER',
 'TERRY MAUPIN',
 'WENDY MAUSOLF']

In [271]:
findall('littlejohn')

['DEWIGHT MAURICE LITTLEJOHN']

In [272]:
findall('buchser')

['SUSAN MAUDE BUCHSER']

In [273]:
dones.extend(findall(' mau'))

In [274]:
findall(' snyd')

['JASON SNYDER',
 'LILI SNYDER',
 'MICHAEL SNYDER',
 'ROY SNYDER',
 'SPENCER ERIC SNYDER',
 'TERRANCE SNYDER']

In [275]:
dones.extend(findall(' snyd'))

In [276]:
findall(' pur')

['BYRON PURCELL',
 'CRAIG PURSER',
 'KATHRYN PURCELL BALCERZAK',
 'LISA BRAWAND PURVIS',
 'MICHAEL PURZYCKI',
 'PAULA PURDY',
 'SALVATORE PURPURA',
 'SANJAY PURI',
 'STARR PURDUE']

In [277]:
findall(' balc')

['KATHRYN PURCELL BALCERZAK', 'MARY ELLEN BALCHUNIS']

In [278]:
dones.extend(findall(' pur'))

In [279]:
findall(' lei')

['ALEXANDER LEIGH BRONSON',
 'AMANDA LEIGH KASZUBA',
 'BRYAN LEIB',
 'CODY LEISTIKOW',
 'CYNTHIA LEIGH APPLEBAUM',
 'DEAN LEIGH FOWLER',
 'ERIC LEIGH KELLER',
 'GLEN LEIBOWITZ',
 'JANE LEIDERMAN',
 'JASON LEINEWEBER',
 'JEFFREY LEIGH DIMIT',
 'JEFFREY LEITER',
 'REBECCA LEIGH HUBBARD',
 'SUSAN LEIVAS-STURNER']

In [280]:
findall('bronson')

['ALEXANDER LEIGH BRONSON']

In [281]:
findall('kasz')

['AMANDA LEIGH KASZUBA']

In [282]:
findall(['cynth', 'apple'])

['CYNTHIA LEIGH APPLEBAUM']

In [283]:
findall('fowler')

['DEAN LEIGH FOWLER',
 'DONALD FOWLER',
 'JERLDEAN FOWLER',
 'SCOTT FOWLER',
 'SEAN FOWLER',
 'TADD FOWLER']

In [284]:
findall('keller')

['CRAIG KELLER',
 'ERIC KELLER',
 'ERIC LEIGH KELLER',
 'LYNNE KELLER',
 'SCOTT KELLER']

In [285]:
cm[cm['tres_nm'] == 'ERIC KELLER']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
9819,C00684282,EDEMOCRATS PAC LLC,ERIC KELLER,70 LINDEN AVE,P O BOX 2134,HADDONFIELD,NJ,08033,U,N,NaN,A,NaN,NaN,NaN,"KELLER, ERIC MR",70 LINDEN AVENUE,"70 LINDEN AVE, HADDONFIELD, NJ"


In [286]:
cm[cm['tres_nm'] == 'ERIC LEIGH KELLER']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
8514,C00666537,PRESS FREEDOM POLITICAL ACTION COMMITTEE,ERIC LEIGH KELLER,902 TURKEY RUN RD,NaN,MCLEAN,VA,22101,U,N,NaN,A,NaN,NaN,NaN,"KELLER, ERIC LEIGH MR.",902 TURKEY RUN ROAD,"902 TURKEY RUN RD, MCLEAN, VA"


In [287]:
findall(' hubbard')

['AARON HUBBARD',
 'ALEXANDER HUBBARD',
 'EDWARD HUBBARD',
 'MALVA HUBBARD',
 'MARY ELIZABETH HUBBARD',
 'REBECCA LEIGH HUBBARD',
 'STEVEN ERICH HUBBARD',
 'TSHOMBE HUBBARD']

In [288]:
dones.extend(findall(' lei'))

In [289]:
findall(' tea')

['DONALD TEAL', 'KATE TEASDALE', 'MATT TEAGARDEN', 'RICHARD TEAMAN']

In [290]:
dones.extend(findall(' tea'))

In [291]:
findall(' rut')

['BARBARA RUTH BELLAR',
 'JANNA RUTLAND',
 'JOHN RUTKAUSKAS',
 'LUCY RUTISHAUSER',
 'STEPHANIE RUTH MAYO',
 'SUSAN RUTT']

In [292]:
findall('bellar')

['BARBARA RUTH BELLAR']

In [293]:
findall(' mayo')

['FREDERICK MAYOCK', 'GREG ADAM MAYON', 'STEPHANIE RUTH MAYO']

In [294]:
dones.extend(findall(' mayo'))

In [295]:
dones.extend(findall(' rut'))

In [296]:
findall(' elli')

['ALLEN ELLISON',
 'AMY ELLIS',
 'BRIAN JAMES ELLISON',
 'CLAIRE ELISABETH ELLIOTT',
 'GORDON ELLIOTT',
 'GREG ELLIOTT',
 'HEIDI ELLIS ROBEY',
 'MICHAEL ELLINGER',
 'MICHAEL ELLIOTT',
 'PAUL ELLIOTT DAHL',
 'SEAN ELLIOTT',
 'SHERRY ELLIS',
 'THERESA ELLINGTON']

In [297]:
findall(' robey')

['HEIDI ELLIS ROBEY']

In [298]:
findall(' dahl')

['DENNIS DAHL', 'LISA DAHLING', 'PAUL ELLIOTT DAHL', 'THOMAS DAHLEN']

In [299]:
dones.extend(findall(' elli'))

In [300]:
findall(' kee')

['BRITTANY KEELS',
 'DARRYL KEENAN SEGARS',
 'DAVID KEEN',
 'HELENE KEELEY',
 'LYNETTE KEETON',
 'MELISSA KEELING',
 'PATRICK VAN KEERBERGEN',
 'PETER KEEFE',
 'THORNTON KEEL']

In [301]:
findall(' segars')

['DARRYL KEENAN SEGARS']

In [302]:
dones.extend(findall(' kee'))

In [303]:
findall(' jan')

['AUSTIN BENNET TR JANSSEN',
 'BEVE JANE THAKHAMHOR',
 'CHUN JANG',
 'DEBORAH JANSEN',
 'ERIC JANUARY',
 'GORAN JANKOVIC',
 'ISABEL JANCI',
 'JOHN JANKOWSKI',
 'JOSHUA JANTZ',
 'MARTHA JANE OSGERBY',
 'MELISSA JANKOWSKI',
 'NICOLE JANES',
 'PAMELA JANE WHARTON',
 'STEVEN JANELLI',
 'TIMOTHY JANISSE',
 'TRACY JANELL TANIS']

In [304]:
findall(' janssen')

['AUSTIN BENNET TR JANSSEN']

In [305]:
findall(' thakh')

['BEVE JANE THAKHAMHOR']

In [306]:
findall('osgerby')

['MARTHA JANE OSGERBY']

In [307]:
findall('wharton')

['CYNTHIA WHARTON', 'PAMELA JANE WHARTON']

In [308]:
findall(' tanis')

['TRACY JANELL TANIS']

In [309]:
dones.extend(findall(' jan'))

In [310]:
findall(' dupr')

['ABBY DUPREE', 'MARGOT DUPRE', 'MARIE DUPREE', 'SUSAN DUPREY']

In [311]:
dones.extend(findall(' dupr'))

In [312]:
findall(' schwartz')

['ANDREW SCHWARTZ',
 'ARTHUR SCHWARTZ',
 'KARL SCHWARTZ',
 'LAURA ANN SCHWARTZ',
 'LAURA SCHWARTZ',
 'MELODIE SCHWARTZ',
 'MICHAEL SCHWARTZ',
 'STEVEN SCHWARTZ']

In [313]:
cm[cm['tres_nm'] == 'LAURA SCHWARTZ']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
5690,C00580415,FASO FOR CONGRESS,LAURA SCHWARTZ,PO BOX 448,NaN,KINDERHOOK,NY,12106,P,H,REP,Q,NaN,PROTECT THE HOUSE,H6NY19169,"SCHWARTZ, LAURA A. MS.",PO BOX 448,"PO BOX 448, KINDERHOOK, NY"
6009,C00602755,LANGE FOR CONGRESS,LAURA SCHWARTZ,PO BOX 481,NaN,SOUTH SALEM,NY,10590,P,H,REP,Q,NaN,NONE,NaN,"SCHWARTZ, LAURA A.",PO BOX 481,"PO BOX 481, SOUTH SALEM, NY"
6684,C00630681,EMPIRE STATE PAC,LAURA SCHWARTZ,PO BOX 98,NaN,SOUTH SALEM,NY,10590,D,N,NaN,T,NaN,NaN,NaN,"SCHWARTZ, LAURA A.",PO BOX 98,"PO BOX 98, SOUTH SALEM, NY"
6808,C00633263,JOHN FASO VICTORY COMMITTEE,LAURA SCHWARTZ,PO BOX 98,NaN,SOUTH SALEM,NY,10590,J,N,NaN,T,NaN,NaN,NaN,"SCHWARTZ, LAURA",PO BOX 98,"PO BOX 98, SOUTH SALEM, NY"
10680,C00694778,NICOLE FOR NEW YORK,LAURA SCHWARTZ,PO BOX 60487,NaN,STATEN ISLAND,NY,10306,P,H,REP,Q,NaN,MALLIOTAKIS VICTORY COMMITTEE,H0NY11078,"SCHWARTZ, LAURA",PO BOX 60487,"PO BOX 60487, STATEN ISLAND, NY"
11496,C00703454,EMPIRE STATE POLITICAL ACTION COMMITTEE,LAURA SCHWARTZ,PO BOX 98,NaN,SOUTH SALEM,NY,10590,U,N,NaN,T,NaN,NaN,NaN,"SCHWARTZ, LAURA",PO BOX 98,"PO BOX 98, SOUTH SALEM, NY"
14159,C00731356,MALLIOTAKIS VICTORY COMMITTEE,LAURA SCHWARTZ,PO BOX 68,NaN,SOUTH SALEM,NY,10590,J,N,NaN,Q,NaN,NONE,NaN,"SCHWARTZ, LAURA A",PO BOX 68,"PO BOX 68, SOUTH SALEM, NY"


In [314]:
cm[cm['tres_nm'] == 'LAURA ANN SCHWARTZ']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
10963,C00697920,GEORGE PHILLIPS FOR CONGRESS,LAURA ANN SCHWARTZ,PO BOX 477,NaN,ENDICOTT,NY,13761,P,H,REP,Q,NaN,NONE,H6NY22098,"SCHWARTZ, LAURA ANN",PO BOX 477,"PO BOX 477, ENDICOTT, NY"


In [315]:
dones.extend(findall(' schwartz'))

In [316]:
findall(' kau')

['ANNE MARIE KAUFMANN',
 'EMILY KAUFMAN',
 'JASON KAUNE',
 'KARL KAUFMANN',
 'MATTHEW KAULANA ING',
 'PHIL KAUFMAN',
 'ROBERT KAUFMANN',
 'RON KAUFMAN',
 'RONALD KAUFMAN',
 'SCOTT KAUFMAN',
 'STEPHEN KAUFMAN']

In [317]:
cm[cm['tres_nm'] == 'RON KAUFMAN']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
10535,C00693093,COMMITTEE ON ARRANGEMENTS FOR THE 2020 REPUBLI...,RON KAUFMAN,310 FIRST ST SE,NaN,WASHINGTON,DC,20003,U,X,REP,M,NaN,REPUBLICAN NATIONAL COMMITTEE,NaN,"KAUFMAN, RON",310 FIRST STREET SE,"310 FIRST ST SE, WASHINGTON, DC"


In [318]:
cm[cm['tres_nm'] == 'RONALD KAUFMAN']

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
41,C00003418,REPUBLICAN NATIONAL COMMITTEE,RONALD KAUFMAN,310 FIRST ST SE,NaN,WASHINGTON,DC,20003,U,Y,REP,M,NaN,NaN,NaN,"KAUFMAN, RONALD C MR",310 FIRST STREET SE,"310 FIRST ST SE, WASHINGTON, DC"
13449,C00723932,ANGIE WINDHAUSER FOR US PRESIDENT,RONALD KAUFMAN,PO BOX 785098,NaN,WINTER GARDEN,FL,34778,P,P,REP,Q,NaN,REPUBLICAN NATIONAL COMMITTEE,NaN,"KAUFMAN, RONALD C",PO BOX 785098,"PO BOX 785098, WINTER GARDEN, FL"


In [319]:
new = 'RONALD KAUFMAN'
dones.append(new)

these = findall(['ron', ' kaufman'])

getall(these, new)

orig len: 2
new len: 3


In [320]:
dones.extend(findall(' kau'))

In [321]:
findall(' riesc')

['JOSE RIESCO']

In [322]:
dones.extend(findall(' riesc'))

In [323]:
findall(' cop')

['ARTHUR COPLESTON',
 'BENJAMIN COPELAND',
 'DONALD COPE',
 'ELIZABETH COPPINGER',
 'MARYAM COPE',
 'RITA COPELAND',
 'VONA COPP']

In [324]:
dones.extend(findall(' cop'))

In [325]:
findall(' bal')

['AJ BALUKOFF',
 'ALEXANDER BALOGA',
 'BRIANA BALESKIE',
 'JAMES BALDINI',
 'JENNIFER BALL',
 'JOHN BALLARD',
 'JONATHAN BALL',
 'JOSHUA BALLARD',
 'KATHRYN PURCELL BALCERZAK',
 'KRYSTAL BALL',
 'LAURA BALDI',
 'MARY ELLEN BALCHUNIS',
 'MATANGI BALA',
 'MATTHEW BALAZIK',
 'MICHAEL BALL',
 'MIKE BALDWIN',
 'PAUL BALASSA',
 'RICHARD BALL',
 'RONALD BALDWIN',
 'SAL BALDENEGRO',
 'STEVE BALLAN',
 'SUSAN BALKENBUSH',
 'TRACEY BALL']

In [326]:
dones.extend(findall(' bal'))

In [327]:
findall(' tall')

['REBECCA TALLENT', 'ROBERT TALLIA', 'VINCENT TALLMAN']

In [328]:
dones.extend(findall(' tall'))

In [329]:
findall(' juk')

['JOEL JUKUS']

In [330]:
dones.extend(findall(' juk'))

In [331]:
findall(' milln')

['MICHAEL MILLNER']

In [332]:
dones.extend(findall(' milln'))

In [333]:
findall(' goode')

['GEORGE GOODE',
 'JOY ALICIA PAULETTE GOODE',
 'KATHLEEN GOODE',
 'MICHAEL GOODE',
 'WARREN GOODE']

In [334]:
dones.extend(findall(' goode'))

In [335]:
findall(' kun')

['CRAIG KUNKLE',
 'DAVE KUNES',
 'HOORIA KUNDI',
 'JOHN KUNITZ',
 'JOHN WILLIAM KUNITZ',
 'LORNA KUNEY',
 'TEKIN KUNT',
 'TRISHA KUNZE']

In [336]:
cm[cm['tres_nm'].apply(lambda x: ('KUNITZ' in x) & ('JOHN' in x))]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
6518,C00626093,THIRD CONGRESSIONAL DISTRICT REPUBLICAN PARTY ...,JOHN KUNITZ,PO 390722,NaN,EDINA,MN,55439,U,X,REP,Q,NaN,NONE,NaN,"KUNITZ, JOHN W MR.",PO 390722,"PO 390722, EDINA, MN"
9390,C00678789,CONSERVATIVES FOR AMERICA,JOHN WILLIAM KUNITZ,6441 BRETTON WAY,NaN,CHANHASSEN,MN,55317,U,X,REP,T,NaN,NaN,NaN,"KUNITZ, JOHN WILLIAM MR",6441 BRETTON WAY,"6441 BRETTON WAY, CHANHASSEN, MN"


In [337]:
new = 'JOHN WILLIAM KUNITZ'
dones.append(new)

these = findall(['kunitz', 'john'])

getall(these, new)

orig len: 1
new len: 2


In [338]:
dones.extend(findall(' kun'))

In [339]:
findall('lefko')

['JON LEFKOWITZ', 'LEFKOWITZ']

In [340]:
cm[cm['tres_nm'].apply(lambda x: 'LEFKOWITZ' in x)]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
4565,C00524355,PATRIOTS FOR ECONOMIC FREEDOM,JON LEFKOWITZ,151 1ST AVE 214,NaN,NEW YORK,NY,10003,U,V,NaN,Q,NaN,NONE,NaN,"LEFKOWITZ, JON",151 1ST AVE 214,"151 1ST AVE 214, NEW YORK, NY"
5037,C00553313,CONSERVATIVE AMERICA NOW PAC,LEFKOWITZ,151 1ST AVE,214,NEW YORK,NY,10003,U,W,NaN,Q,NaN,NONE,NaN,"LEFKOWITZ, J",151 1ST AVE,"151 1ST AVE, NEW YORK, NY"
5416,C00570903,RESTORE AMERICAN FREEDOM AND LIBERTY,LEFKOWITZ,151 1ST AVE 214,NaN,NEW YORK,NY,10003,U,V,NaN,Q,NaN,NONE,NaN,"LEFKOWITZ, J",151 1ST AVE 214,"151 1ST AVE 214, NEW YORK, NY"
5557,C00575118,CONSERVATIVE LIBERTY COALITION,LEFKOWITZ,151 1ST AVE,214,NEW YORK,NY,10003,U,O,NaN,Q,NaN,NONE,NaN,"LEFKOWITZ, J",151 1ST AVE,"151 1ST AVE, NEW YORK, NY"
5792,C00584482,RESTORE THE CONSTITUTION COALITION,LEFKOWITZ,151 1ST AVE,214,NEW YORK,NY,10003,U,V,NaN,Q,NaN,NONE,NaN,"LEFKOWITZ, J",151 1ST AVE,"151 1ST AVE, NEW YORK, NY"
6010,C00602797,CONSERVATIVE FREEDOM FIGHTERS,LEFKOWITZ,151 1ST AVE 214,NaN,NEW YORK,NY,10003,U,V,NaN,Q,NaN,NONE,NaN,"LEFKOWITZ, J",151 1ST AVE 214,"151 1ST AVE 214, NEW YORK, NY"
6012,C00602813,SAVE THE AMERICAN WAY,LEFKOWITZ,151 1ST AVE 214,NaN,NEW YORK,NY,10003,U,V,NaN,Q,NaN,NONE,NaN,"LEFKOWITZ, J",151 1ST AVE 214,"151 1ST AVE 214, NEW YORK, NY"


In [341]:
new = 'JON LEFKOWITZ'
dones.append(new)

these = findall('lefkowitz')

getall(these, new)

orig len: 1
new len: 7


In [342]:
findall(' sif')

['TIM SIFERT']

In [343]:
dones.extend(findall(' sif'))

In [344]:
findall(' vand')

['CRAIG VANDERVEER',
 'DAVID VANDER VEEN',
 'DIRK VANDONGEN',
 'ELIZABETH ANN VANDERWAY',
 'JASON VANDEVER',
 'JUSTIN VANDERBILT',
 'MARK ALLEN VANDEWEGE',
 'MIKE VANDERWEY',
 'WILLIAM VANDERBROOK']

In [345]:
dones.extend(findall(' vand'))

In [346]:
findall(' changkyu')

['COALITION FOR ELECTO CHANGKYU KIM', 'JASON CHANGKYU KIM']

In [347]:
new = 'JASON CHANGKYU KIM'
dones.append(new)

these = findall(' changkyu')

getall(these, new)

orig len: 6
new len: 11


In [348]:
findall(['ash', 'newman'])

['ASHLEY NEWMAN']

In [349]:
dones.extend(findall(['ash', 'newman']))

In [350]:
findall(' rogers')

['BRETT ROGERS',
 'CLAIR CHISLER ROGERS',
 'EMORY ROGERS',
 'FRANK ROGERS',
 'GEORGE ROGERS',
 'GINGER GREENE ROGERS',
 'GLEN ROGERS',
 'GRACE ROGERS',
 'KEVIN ROGERS',
 'MARCIA MATHISON ROGERS',
 'MICHAEL ROGERS',
 'STEVEN ROGERS',
 'TAMMY ROGERS',
 'TERRANCE ROGERS']

In [351]:
dones.extend(findall(' rogers'))

In [352]:
findall(' kennedy')

['ALEXANDER KENNEDY',
 'ANGELA KENNEDY',
 'BUCKY KENNEDY',
 'CASTLEN KENNEDY',
 'CHRIS KENNEDY',
 'CHRISTOPHER KENNEDY',
 'JEFF GRANT KENNEDY',
 'JESSICA KENNEDY',
 'JESSIE KENNEDY WEISS',
 'KATHERINE KENNEDY',
 'KATIE KENNEDY',
 'KATRINA KENNEDY',
 'KEVIN KENNEDY',
 'KYLE KENNEDY',
 'MANDY KENNEDY',
 'MICHAEL KENNEDY',
 'MICHAEL WARREN KENNEDY',
 'RICHARD KENNEDY',
 'ROBERT KENNEDY',
 'WILLIAM GLENN KENNEDY']

In [353]:
cm[cm['tres_nm'].apply(lambda x: x in ['CHRIS KENNEDY', 'CHRISTOPHER KENNEDY'])]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
1235,C00222455,CITIZENS FOR JOE KENNEDY 1988,CHRISTOPHER KENNEDY,C/O S KIDDER HEMENWAY & BARNES,75 STATE STREET,BOSTON,MA,02109,P,H,DEM,Q,NaN,NaN,H6MA08141,"KENNEDY, CHRISTOPHER","C/O S. KIDDER, HEMENWAY & BARNES","C/O S KIDDER HEMENWAY & BARNES, BOSTON, MA"
7798,C00655621,CHRIS KENNEDY FOR COLORADO,CHRIS KENNEDY,2514 FILMORE AVE,NaN,GRAND JUNCTION,CO,81505,P,H,DEM,A,NaN,NaN,H8CO03200,"KENNEDY, CHRIS",2514 FILMORE AVE,"2514 FILMORE AVE, GRAND JUNCTION, CO"


In [354]:
cm[cm['tres_nm'].apply(lambda x: x in ['KATIE KENNEDY', 'KATHERINE KENNEDY'])]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
4891,C00546325,TAKING AN INDEPENDENT PERSPECTIVE TOGETHER FOR...,KATHERINE KENNEDY,2318 CURTIS ST,NaN,DENVER,CO,80205,D,N,NaN,Q,NaN,NaN,NaN,KATHERINE KENNEDY,2318 CURTIS STREET,"2318 CURTIS ST, DENVER, CO"
5154,C00558759,PDC ENERGY INC PAC,KATHERINE KENNEDY,1775 SHERMAN ST,NaN,DENVER,CO,80203,U,Q,NaN,Q,C,NaN,NaN,"KENNEDY, KATHERINE",1775 SHERMAN STREET SUITE 3000,"1775 SHERMAN ST, DENVER, CO"
5260,C00564435,PRINCIPLED CONSERVATIVES OF COLORADO,KATIE KENNEDY,600 17TH ST,NaN,DENVER,CO,80202,U,O,NaN,T,NaN,NaN,NaN,KATIE KENNEDY,600 17TH STREET SUITE 2800 SOUTH,"600 17TH ST, DENVER, CO"
5703,C00580894,VAIL RESORTS EMPLOYEE POLITICAL ACTION COMMITTEE,KATIE KENNEDY,390 INTERLOCKEN CRESCENT,NaN,BROOMFIELD,CO,80021,U,Q,NaN,Q,C,NaN,NaN,"KENNEDY, KATIE",390 INTERLOCKEN CRESCENT,"390 INTERLOCKEN CRESCENT, BROOMFIELD, CO"
8800,C00670505,COMMITTEE FOR BARRINGTON,KATIE KENNEDY,745 S MILLER ST,NaN,LAKEWOOD,CO,80226,P,H,REP,T,NaN,NaN,H8CO07094,"KENNEDY, KATIE",745 S. MILLER STREET,"745 S MILLER ST, LAKEWOOD, CO"
9506,C00680306,BARRIS FOR CONGRESS,KATIE KENNEDY,24853 CHRIS DR,NaN,EVERGREEN,CO,80439,P,H,LIB,T,NaN,NaN,H8CO02269,"KENNEDY, KATIE",24853 CHRIS DRIVE,"24853 CHRIS DR, EVERGREEN, CO"
9681,C00682559,PROTECTING OUR CONSTITUTION,KATIE KENNEDY,5910 S UNIVERSITY BLVD C18 NO 254,NaN,GREENWOOD VILLAGE,CO,80121,U,O,NaN,Q,NaN,NONE,NaN,"KENNEDY, KATIE",5910 S. UNIVERSITY BLVD C18 NO 254,"5910 S UNIVERSITY BLVD C18 NO 254, GREENWOOD V..."
12856,C00717868,HOUSE CD6,KATIE KENNEDY,992 S 4TH AVE,"SUITE 100, #440",BRIGHTON,CO,80601,P,H,REP,Q,NaN,NONE,H0CO06119,"KENNEDY, KATIE",992 S. 4TH AVE,"992 S 4TH AVE, BRIGHTON, CO"


In [355]:
new = 'KATHERINE KENNEDY'
dones.append(new)

these = findall(['kat', 'kennedy'])

getall(these, new)

orig len: 2
new len: 9


In [356]:
cm[cm['tres_nm'].apply(lambda x: x in [
    'MICHAEL KENNEDY', 
    'MICHAEL WARREN KENNEDY', 
])]

,cmte_id,cmte_nm,tres_nm,cmte_st1,cmte_st2,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_tp,cmte_pty_affiliation,cmte_filing_freq,org_tp,connected_org_nm,cand_id,orig_tres_nm,orig_cmte_st1,address
3808,C00477299,"VMWARE, INC. POLITICAL ACTION COMMITTEE (VMWAR...",MICHAEL KENNEDY,3401 HILLVIEW AVE,NaN,PALO ALTO,CA,94304,U,Q,NaN,M,C,NaN,NaN,"KENNEDY, MICHAEL",3401 HILLVIEW AVE,"3401 HILLVIEW AVE, PALO ALTO, CA"
11912,C00708073,RONDA KENNEDY FOR CONGRESS 2020,MICHAEL WARREN KENNEDY,4607 LAKEVIEW CANYON RD,#522,WESTLAKE VILLAGE,CA,91361,P,H,REP,Q,NaN,NONE,H0CA26145,"KENNEDY, MICHAEL WARREN",4607 LAKEVIEW CANYON RD.,"4607 LAKEVIEW CANYON RD, WESTLAKE VILLAGE, CA"


In [357]:
dones.extend(findall(' kennedy'))

In [358]:
findall(['mel', 'allen'])

['KYLE ALLEN HIMMELWRIGHT', 'MELISSA ALLEN']

In [359]:
findall(' himmel')

['KYLE ALLEN HIMMELWRIGHT', 'MARC HIMMELSTEIN']

In [360]:
dones.extend(findall(' himmel'))
dones.extend(findall(['mel', 'allen']))

In [361]:
findall(' reis')

['ADAM REISER',
 'ALAN REISCHE',
 'BRANDON REISER',
 'BRANDON REISINGER',
 'DANIEL REISTETER',
 'MICHELE REISNER']

In [362]:
dones.extend(findall(' reis'))

In [363]:
findall(' doz')

['JAMES DOZIER', 'JOSEPH DOZIER', 'JULIE DOZIER']

In [364]:
dones.extend(findall(' doz'))

In [365]:
findall(' roz')

['ADAM ROZANSKY', 'COSTANTINO ROZZO']

In [366]:
dones.extend(findall(' roz'))

In [367]:
findall(' patt')

['BENTLEY PATTERSON',
 'CHRIS PATTON',
 'EDEN SOLAR PATTERSON',
 'EZEKIEL PATTERSON',
 'JAMES PATTON',
 'ROBERT PATTISON',
 'SCOTT PATTON',
 'STEPHEN PATTERSON',
 'THERESA PATTARA',
 'VANCE PATTERSON',
 'WILLIAM RALPH PATTERSON']

In [368]:
dones.extend(findall(' patt'))

The work is never done...

In [369]:
# checks which have already been deduped and does not display those
mask = [False if cm.loc[i, 'tres_nm'] in dones else True for i in list(cm.index)]
cm.loc[mask, 'tres_nm'].value_counts()[cm.loc[mask, 'tres_nm'].value_counts() > 1][:20]

RUSSELL MILLER         6
MATTHEW YUSKEWICH      6
CHRISTOPHER WOODFIN    5
JEAN PAUL BRADSHAW     5
EYTAN LAOR             5
JEFFREY CARSON         5
MARIA WOJCIECHOWSKI    5
SHELLY MOSKWA          5
YOLANDA MIRANDA        5
SCOTT MACKENZIE        5
BEVERLY SHEA           5
REBECCA OLSON          5
FLORA YIN              5
ROBERT PIARO           5
LYSA RAY               5
SCOTT FALMLEN          5
JESSICA BROUCKAERT     5
JASON BENNETT          5
ALLISON MURRAY         5
REBECCA LUBY           5
Name: tres_nm, dtype: int64

In [370]:
dupename = list(cm['cmte_nm'].value_counts()[cm['cmte_nm'].value_counts() > 1].index)

for dupe in dupename:
    lil_cm = cm.loc[cm['cmte_nm'] == dupe, [
        'cmte_id',
        'tres_nm',
        'cmte_st1',
        'cmte_st',
    ]].sort_values(['cmte_st'])
    print(dupe)
    print(lil_cm)
    print('\n')

COLLINS FOR CONGRESS
        cmte_id               tres_nm             cmte_st1 cmte_st
4255  C00502039  CARROLL BRIAN DANIEL          PO BOX 1295      GA
4848  C00544684         JULIE COLLINS           PO BOX 855      GA
6156  C00611137    CATHERINE ANN PAGE  3045 DARIEN PARK DR      GA
1901  C00335695          MARY COLLINS  9660 COBBLESTONE DR      NY
4509  C00520379       JOCELYN JAKUBUS           PO BOX 386      NY
4523  C00521641       CYNTHIA GIFFORD    176 SYCAMORE LANE      RI


SCHNEIDER FOR CONGRESS
        cmte_id         tres_nm          cmte_st1 cmte_st
2308  C00374751   JAN SCHNEIDER  227 SEAGULL LANE      FL
4141  C00495952  DEBORAH KARTON       PO BOX 1318      IL
8974  C00672956   BRADLEY CRATE        PO BOX 317      MA


WINNING FOR WOMEN INC.
         cmte_id                tres_nm                     cmte_st1 cmte_st
14793  C30002893  (no treasurer listed)  1025 THOMAS JEFFERSON ST NW      DC
15147  C90017609  (no treasurer listed)  1025 THOMAS JEFFERSON ST NW      

ROBERT CROOK FOR CONGRESS
         cmte_id          tres_nm        cmte_st1 cmte_st
9727   C00683169  MARIANO VERRICO  3 LENFELL LANE      NJ
14050  C00730192  MARIANO VERRICO      PO BOX 635      NJ


JAKE AUCHINCLOSS FOR CONGRESS
         cmte_id          tres_nm       cmte_st1 cmte_st
13204  C00721431  ANTHONY LOGALBO  PO BOX 600698      MA
13205  C00721449   BRENDAN GALVIN  PO BOX 600698      MA


SANTOS FOR CONGRESS
         cmte_id         tres_nm        cmte_st1 cmte_st
9008   C00673467  KENNETH NOWELL      53 PECK RD      CT
13540  C00724872  KRISTAN SANTOS  108 HICKORY LN      OH


STEVEN CASTELLANO FOR US CONGRESS
         cmte_id                 tres_nm             cmte_st1 cmte_st
7238   C00641266  STEVEN DION CASTELLANO  6601 BLUE OAKS BLVD      CA
14382  C00733626  STEVEN DION CASTELLANO      1114 W 10TH AVE      WA


FREEDOM PAC
         cmte_id       tres_nm            cmte_st1 cmte_st
3865   C00482703  PAUL KILGORE  824 S MILLEDGE AVE      GA
12915  C00718460  YEHUDA R

In [371]:
# show all cmte_nm & tres_nm for committees sharing a street address
multiadd = list(cm['address'].value_counts()[cm['address'].value_counts() > 10].index)

for add in multiadd:
    lil_cm = cm.loc[cm['address'] == add, ['tres_nm', 'cmte_nm']]\
    .sort_values(['tres_nm', 'cmte_nm'], ascending = [True, True]).reset_index(drop = True)
    print(add)
    print(lil_cm.to_csv(None))
    print('\n')

228 S WASHINGTON ST, ALEXANDRIA, VA
,tres_nm,cmte_nm
0,DAVID SATTERFIELD,ARKANSAS HORIZON
1,DAVID SATTERFIELD,BE OF GOOD CHEER PAC
2,DAVID SATTERFIELD,BIGGS FOR CONGRESS
3,DAVID SATTERFIELD,BUILDING INNOVATIVE GOOD GOVERNMENT SOLUTIONS PAC
4,DAVID SATTERFIELD,CONSERVATIVE RESULTS
5,DAVID SATTERFIELD,CONSTITUTIONAL CONSERVATIVES
6,DAVID SATTERFIELD,ELBERT GUILLORY'S AMERICA
7,DAVID SATTERFIELD,FREE AT LAST PAC
8,DAVID SATTERFIELD,FREEDOM FUND FOR AMERICA'S FUTURE INC
9,DAVID SATTERFIELD,FRIENDS OF LENAR
10,DAVID SATTERFIELD,GRAIN AND FEED PAC
11,DAVID SATTERFIELD,HOOSIER HORIZON
12,DAVID SATTERFIELD,JOHN CATS POLITICAL ACTION COMMITTEE
13,DAVID SATTERFIELD,KEYSTONE PRIORITIES
14,DAVID SATTERFIELD,MACK PAC
15,DAVID SATTERFIELD,NATIONAL HORIZON
16,DAVID SATTERFIELD,PATRIOT PROSPERITY POLITICAL ACTION COMMITTEE
17,DAVID SATTERFIELD,POLICE ACTION FUND
18,DAVID SATTERFIELD,REDEMPTION PAC
19,DAVID SATTERFIELD,RJC VICTORY FUND
20,DAVID SATTERFIELD,SAFEGUARDING AMERICAN LIBERTIES MORALS & OPPOR

777 S FIGUEROA ST, LOS ANGELES, CA
,tres_nm,cmte_nm
0,(no treasurer listed),HUNTINGTON BEACH RESIDENTS FOR SAFE NEIGHBORHOODS IN SUPPORT OF BILLY O'CONNELL FOR HUNTINGTON BEACH CITY COUNCIL 2018
1,(no treasurer listed),KRC IN ACTION
2,ALEXANDER WARREN,PORTER VICTORY FUND
3,BIANCA RZESLAWSKI,AMMAR CAMPA-NAJJAR VICTORY FUND
4,CHARLOTTE WHITE,DEMOCRATIC VICTORY FUND
5,CLIFF CHENFELD,CHANGE IS COMING
6,EDWARD KURTZ,COURAGE CAMPAIGN SUPER PAC
7,IDA YARBROUGH,AFRICAN AMERICAN VOTER REGISTRATION EDUCATION AND PARTICIPATION PROJECT
8,JAMIE KANTROWITZ,LOS ANGELES WOMEN'S GIVING COLLECTIVE PAC
9,JANICE MARTIN,AFRICAN AMERICAN VOTER REGISTRATION EDUCATION AND PARTICIPATION PROJECT - FEDERAL
10,JESS DURFEE,GEORGETTE GOMEZ FOR CONGRESS
11,JESS DURFEE,GEORGETTE GOMEZ VICTORY FUND
12,KAREN WINGARD,LOS ANGELES COUNTY DEMOCRATIC CENTRAL COMMITTEE
13,KEN SULZER,FRONTLINE USA
14,LORI ALEXANDER,HEALTH JUSTICE PAC
15,MARC WINGER,CHRISTY SMITH FOR CONGRESS
16,MARC WINGER,CHRISTY SMITH VICTORY FUND
17,MARK R

1787 TRIBUTE RD, SACRAMENTO, CA
,tres_nm,cmte_nm
0,(no treasurer listed),SAFETY FOR ALL YES ON PROP. 63 NEWSOM BALLOT MEASURE COMMITTEE
1,KIM LUTZ,HUMANITY FORWARD FUND
2,SAL ROSSELLI,NATIONAL UNION OF HEALTHCARE WORKERS FEDERAL COMMITTEE ON POLITICAL EDUCATION
3,SHAWNDA DEANE,4WARDSTEPS
4,SHAWNDA DEANE,AISHA WAHAB FOR CONGRESS
5,SHAWNDA DEANE,CA-BAM PAC
6,SHAWNDA DEANE,CANNING FOR CONGRESS
7,SHAWNDA DEANE,ESMERALDA SORIA FOR CONGRESS
8,SHAWNDA DEANE,FIGHT BACK CALIFORNIA
9,SHAWNDA DEANE,FORWARD CALIFORNIA
10,SHAWNDA DEANE,FRANCO FOR CONGRESS
11,SHAWNDA DEANE,LOS ANGELES COUNTY YOUNG DEMOCRATS FEDERAL PAC
12,SHAWNDA DEANE,MARIANNE WILLIAMSON FOR CONGRESS
13,SHAWNDA DEANE,MARIANNE WILLIAMSON FOR PRESIDENT
14,SHAWNDA DEANE,MARK GAMBA FOR CONGRESS
15,SHAWNDA DEANE,NEED TO IMPEACH



1201 PENNSYLVANIA AVE, WASHINGTON, DC
,tres_nm,cmte_nm
0,CLAY PARKER DAVIS,HELP AMERICA'S LEADERS POLITICAL ACTION COMMITTEE (HALPAC)
1,DAVID MASON,"STANLEY BLACK & DECKER, INC. POLITICAL ACTION COMMITTEE"
2,K

2350 KERNER BLVD, SAN RAFAEL, CA
,tres_nm,cmte_nm
0,CHRISTOPHER SKINNELL,PERFORMANT FINANCIAL CORPORATION POLITICAL ACTION COMMITTEE
1,JASON KAUNE,BIOMARIN PHARMACEUTICAL INC. POLITICAL ACTION COMMITTEE AKA BIOMARIN PAC
2,JASON KAUNE,"GEP ADMINISTRATIVE SERVICES, INC. EMPLOYEE PAC"
3,JASON KAUNE,NEW MAJORITY FEDERAL PAC
4,JASON KAUNE,"PROFESSIONAL SERVICE INDUSTRIES, INC. POLITICAL ACTION COMMITTEE (PSI PAC)"
5,JASON KAUNE,"PROSPECT MEDICAL HOLDINGS, INC. POLITICAL ACTION COMMITTEE (PROSPECTPAC)"
6,MICHAEL COLUMBO,INVENTOR RIGHTS PAC
7,MICHAEL COLUMBO,OUTFRONT MEDIA INC POLITICAL ACTION COMMITTEE (OFPAC)
8,MICHAEL COLUMBO,"WOODWARD, INC. FOR EFFECTIVE GOVERNMENT"
9,STEVEN LUCAS,MGM RESORTS INTERNATIONAL  PAC
10,STEVEN LUCAS,"SWITCH, INC. PAC"
11,VIGO NIELSEN,"FARMERS GROUP, INC., FARMERS INSURANCE EXCHANGE, FIRE INSURANCE EXCHANGE AND TRUCK INSURANCE EXCHANGE POLITICAL ACTION COMMITTEE AKA FARMERS INSURANCE PAC"



430 SOUTH CAPITOL ST SE, WASHINGTON, DC
,tres_nm,cmte_nm
0,JOSEPH SMOLS

### Other columns

In [372]:
cmtetp_dict = {
    'C' : 'cmte_CommunicationCost',
    'D' : 'cmte_DelegateCommittee',
    'E' : 'cmte_ElectioneeringComms',
    'H' : 'cmte_House',
    'I' : 'cmte_IndependentExpenditor',
    'S' : 'cmte_Senate',
    'N' : 'cmte_PACnonqual',
    'Q' : 'cmte_PACqual',
    'Y' : 'cmte_Partyqual',
    'O' : 'cmte_SuperPAC',
    'P' : 'cmte_Presidential',
    'U' : 'cmte_Singlecand',
    'V' : 'cmte_PACnoncontrib_nonqual',
    'W' : 'cmte_PACnoncontrib_qual',
    'X' : 'cmte_Partynonqual',
    np.nan : np.nan,
}

cm['cmte_tp'] = [cmtetp_dict[x] for x in cm['cmte_tp']]
cm['cmte_tp'].value_counts()

cmte_House                    4374
cmte_PACqual                  3121
cmte_PACnonqual               2888
cmte_SuperPAC                 1633
cmte_Presidential              943
cmte_Senate                    828
cmte_IndependentExpenditor     449
cmte_PACnoncontrib_nonqual     297
cmte_Partynonqual              256
cmte_Partyqual                 247
cmte_Singlecand                 81
cmte_ElectioneeringComms        63
cmte_PACnoncontrib_qual         60
cmte_CommunicationCost          48
Name: cmte_tp, dtype: int64

In [373]:
cmtedsgn_dict = {
    'A' : 'Authorized by a candidate',
    'B' : 'Lobbyist/Registrant PAC',
    'D' : 'Leadership PAC',
    'J' : 'Joint fundraiser',
    'P' : 'Principal campaign committee',
    'U' : 'Unauthorized',
    np.nan : np.nan,
}

cm['cmte_dsgn'] = [cmtedsgn_dict[x] for x in cm['cmte_dsgn']]
cm['cmte_dsgn'].value_counts()

Unauthorized                    6504
Principal campaign committee    5746
Lobbyist/Registrant PAC         1339
Joint fundraiser                 834
Leadership PAC                   644
Authorized by a candidate        218
Name: cmte_dsgn, dtype: int64

In [374]:
cm['cmte_pty_affiliation'].fillna('UNK', inplace = True)
cm.loc[cm['cmte_pty_affiliation'] == '.', 'cmte_pty_affiliation'] = 'UNK'
cm.loc[cm['cmte_pty_affiliation'] == 'UKN', 'cmte_pty_affiliation'] = 'UNK'
cm.loc[cm['cmte_pty_affiliation'] == '  0', 'cmte_pty_affiliation'] = 'UNK'
cm.loc[cm['cmte_pty_affiliation'] == 'UN', 'cmte_pty_affiliation'] = 'UNK'
cm['cmte_pty_affiliation'].value_counts()

UNK    8764
DEM    2866
REP    2545
IND     347
LIB     152
       ... 
TX        1
GOP       1
PPO       1
FLP       1
WRI       1
Name: cmte_pty_affiliation, Length: 76, dtype: int64

In [375]:
cm['cmte_filing_freq'].value_counts()

Q    11465
T     1632
M     1455
A      740
Name: cmte_filing_freq, dtype: int64

In [376]:
orgtp_dict = {
    'C' : 'org_Corporation',
    'H' : 'org_HnotonFECsite',
    'I' : 'org_InotonFECsite',
    'L' : 'org_Labor organization',
    'M' : 'org_Membership organization',
    'T' : 'org_Trade association',
    'V' : 'org_Cooperative',
    'W' : 'org_Corporation without capital stock',
    np.nan : np.nan,
}

cm['org_tp'] = [orgtp_dict[x] for x in cm['org_tp']]
cm['org_tp'].value_counts()

org_Corporation                          1616
org_Trade association                     714
org_Membership organization               451
org_Labor organization                    306
org_Corporation without capital stock      89
org_Cooperative                            46
org_InotonFECsite                           4
org_HnotonFECsite                           2
Name: org_tp, dtype: int64

In [377]:
cm['connected_org_nm'].value_counts()[:10]

NONE                                                                  5920
TAKE BACK THE HOUSE 2020                                                62
DOLLARS FOR DEMOCRATS                                                   14
REPUBLICANS INSPIRING SUCCESS & EMPOWERMENT PROJECT (RISE PROJECT)      10
REPUBLICAN PARTY OF TEXAS                                                9
TAKE BACK THE HOUSE CALIFORNIA 2020                                      8
REPUBLICAN PARTY OF VIRGINIA INC                                         7
NRSC TARGETED STATE VICTORY                                              7
SERVE AMERICA VICTORY FUND                                               6
DEMOCRATIC PARTY OF VIRGINIA                                             6
Name: connected_org_nm, dtype: int64

In [378]:
cm.loc[cm['connected_org_nm'] == 'NONE', 'connected_org_nm'] = np.nan

In [379]:
cm['connected_org_nm'].value_counts()[:10]

TAKE BACK THE HOUSE 2020                                              62
DOLLARS FOR DEMOCRATS                                                 14
REPUBLICANS INSPIRING SUCCESS & EMPOWERMENT PROJECT (RISE PROJECT)    10
REPUBLICAN PARTY OF TEXAS                                              9
TAKE BACK THE HOUSE CALIFORNIA 2020                                    8
NRSC TARGETED STATE VICTORY                                            7
REPUBLICAN PARTY OF VIRGINIA INC                                       7
DEMOCRATIC GRASSROOTS VICTORY FUND                                     6
DEMOCRATIC PARTY OF VIRGINIA                                           6
CALIFORNIA REPUBLICAN PARTY                                            6
Name: connected_org_nm, dtype: int64

In [380]:
# sorry these print ugly but it's to display them unabbreviated
treses = list(cm['tres_nm'].value_counts()[cm['tres_nm'].value_counts() > 10].index)

for tres in treses:
    if tres != '(no treasurer listed)':
        lil_cm = cm[cm['tres_nm'] == tres]
        lil_cm = lil_cm[[
            'cmte_id',
            'cand_id',
            'cmte_nm',

        ]].sort_values(['cand_id']).reset_index(drop = True)
        ptys = sorted(list(set(cm.loc[lil_cm.index, 'cmte_pty_affiliation'])))
        print(''.join([tres, ', treasurer']))
        print(len(lil_cm), 'committees')
        print('parties:', str(ptys))
        print(lil_cm.to_csv(None))
        print('\n\n')

PAUL KILGORE, treasurer
164 committees
parties: ['CRV', 'DEM', 'NAT', 'NNE', 'REP', 'UNK']
,cmte_id,cand_id,cmte_nm
0,C00714337,H0AL02137,WILL DISMUKES FOR CONGRESS
1,C00725689,H0AZ04535,ANNE MARIE WARD FOR CONGRESS
2,C00732966,H0FL03191,GAVIN ROLLINS FOR CONGRESS
3,C00731711,H0FL19197,HEATHER FITZENHAGEN FOR CONGRESS
4,C00716225,H0FL26028,BLANCO FOR CONGRESS
5,C00707612,H0GA06184,"NICOLE RODDEN FOR CONGRESS, INC."
6,C00569079,H0GA07125,HICE FREEDOM FUND
7,C00544445,H0GA07125,JODY HICE FOR CONGRESS
8,C00703504,H0GA07224,HOMRICH FOR CONGRESS
9,C00544510,H0GA08099,AUSTIN SCOTT VICTORY FUND
10,C00462556,H0GA09030,GRAVES FOR CONGRESS
11,C00501866,H0GA09030,TEAM GRAVES
12,C00735761,H0GA14071,"KEVIN COOKE FOR CONGRESS, INC."
13,C00703488,H0IL06052,CITIZENS FOR EVELYN
14,C00707539,H0IL10351,VAL FOR CONGRESS
15,C00458877,H0IL11052,KINZINGER FOR CONGRESS
16,C00467522,H0IL14080,RANDY HULTGREN FOR CONGRESS
17,C00703736,H0IL14163,GRADEL FOR CONGRESS
18,C00716498,H0IL17083,ESTHER FOR CONGRESS
19,C0

LAURA MATTHEWS, treasurer
24 committees
parties: ['DEM', 'NAT', 'NNE', 'REP', 'UNK']
,cmte_id,cand_id,cmte_nm
0,C00660084,S0CA00199,CALIFORNIA SENATE VICTORY 2018
1,C00672857,S0CA00199,WOMEN ON THE ROAD CALIFORNIA 2018
2,C00685057,S0CA00199,WOMEN ON THE ROAD 2018
3,C00585737,S0CO00211,COLORADO SENATE VICTORY 2016
4,C00582817,S6IL00292,ILLINOIS VICTORY 2016
5,C00653857,S6MN00267,MINNESOTA SENATE VICTORY 2018
6,C00641084,S6MO00305,MCCASKILL SENATE VICTORY 2018
7,C00682260,S6PA00217,PENNSYLVANIA SENATE VICTORY 2018
8,C00618520,S6PA00266,PENNSYLVANIA SENATE 2016
9,C00631796,S8FL00166,FLORIDA SENATE RECOUNT 2018
10,C00636191,S8MI00281,MICHIGAN SENATE VICTORY 2018
11,C00672840,S8NV00156,WOMEN ON THE ROAD SEATTLE 2018
12,C00582940,S8WI00026,FEINGOLD VICTORY FUND
13,C00462333,,HOUSE SENATE VICTORY FUND
14,C00599506,,FLORIDA SENATE 2016
15,C00626069,,NEVADA LEGACY FUND
16,C00698548,,SENATE IMPACT 2020
17,C00701789,,MICHIGAN SENATE VICTORY 2020
18,C00708057,,"SENATE IMPACT: AZ, IA & NM"
19,C0071

CARY DAVIDSON, treasurer
12 committees
parties: ['DEM', 'NNE', 'UNK']
,cmte_id,cand_id,cmte_nm
0,C00019653,,EDISON INTERNATIONAL PAC
1,C00282038,,"SONY PICTURES ENTERTAINMENT, INC. PAC"
2,C00390633,,SCHOOLSFIRST FEDERAL CREDIT UNION EMPLOYEES FEDERAL PAC
3,C00392464,,UNIVERSAL MUSIC GROUP EMPLOYEE ACTION FUND
4,C00416636,,"STAMPS.COM, INC. PAC"
5,C00417139,,KB HOME AMERICAN DREAM PAC
6,C00430066,,KINECTA FEDERAL CREDIT UNION FEDERAL PAC
7,C00435735,,UNIVISION COMMUNICATIONS INC. POLITICAL ACTION COMMITTEE
8,C00493635,,EQUALITY CALIFORNIA FEDERAL PAC
9,C00555367,,ASHTON WOODS CORP. HOMES PAC
10,C00680561,,SONY MUSIC ENTERTAINMENT EMPLOYEE ACTION FUND
11,C00701797,,EQUALITY CALIFORNIA VOTES




GEMMA MARTIN, treasurer
12 committees
parties: ['DEM', 'NNE', 'UNK']
,cmte_id,cand_id,cmte_nm
0,C00722132,H0MA04200,CAVELL FOR CONGRESS
1,C00541888,H4MA05084,KATHERINE CLARK FOR CONGRESS
2,C00512970,S0MA00208,KENNEDY FOR MASSACHUSETTS
3,C00717108,S0MA00208,KENNEDY FOR MASSACHUSETTS
4,C00380733,,MA

In [381]:
cm.to_csv('data/03/committees_03a.csv', index = False)